# 지역난방 열수요 예측: 완전한 고도화된 스태킹 앙상블

## 모델링 전략
- **구성**: 3개 규모 그룹 × 2개 계절 = 6개 모델
- **스태킹**: Prophet + CatBoost + LSTM + Ridge 메타모델
- **최적화**: 모든 모델에 Optuna + 3-Fold CV
- **총 모델**: 18개 + 6개 메타모델 = 24개
- **재현성**: 완전한 시드 고정

In [1]:
# Google Colab 환경 확인 및 패키지 설치
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Google Colab 환경에서 실행 중...")
    !pip install catboost prophet torch optuna statsmodels holidays pmdarima scikit-learn==1.3.0 --quiet
    from google.colab import files, drive
    print("패키지 설치 완료!")
else:
    print("로컬 환경에서 실행 중...")

로컬 환경에서 실행 중...


In [2]:
# 완전한 재현성을 위한 시드 고정
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 라이브러리 import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm
import pickle
import holidays
import json
import os
import copy

In [3]:
import os
import warnings
import logging

# Prophet/Stan 로그 완전 차단
os.environ['CMDSTAN_LOGGER'] = 'ERROR'
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

# 특정 로거들 비활성화
for logger_name in ['prophet', 'cmdstanpy', 'pystan']:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)
    logging.getLogger(logger_name).disabled = True

In [4]:
# 머신러닝 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit, KFold
import catboost as cb
from catboost import CatBoostRegressor

# PyTorch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Prophet
try:
    from prophet import Prophet

except ImportError:
    print("Prophet 설치 필요")
    Prophet = None

# Optuna
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ARIMA
try:
    from pmdarima import auto_arima
    from statsmodels.tsa.arima.model import ARIMA
except ImportError:
    print("pmdarima 설치 필요")
    auto_arima = None

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

plt.rcParams['figure.figsize'] = (12, 6)
print("라이브러리 로드 완료! (시드 고정으로 재현성 보장)")

디바이스: cpu
라이브러리 로드 완료! (시드 고정으로 재현성 보장)


In [5]:
# Huber Loss 함수 정의
def huber_loss(y_true, y_pred, delta=1.0):
    """
    Huber Loss 계산
    delta보다 작은 오차에는 제곱 손실, 큰 오차에는 선형 손실 적용
    """
    residual = np.abs(y_true - y_pred)
    condition = residual <= delta
    squared_loss = 0.5 * residual**2
    linear_loss = delta * residual - 0.5 * delta**2
    return np.where(condition, squared_loss, linear_loss).mean()

def evaluate_predictions(y_true, y_pred, delta=1.0):
    """RMSE와 Huber Loss를 동시에 계산"""
    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Huber Loss 계산
    residual = np.abs(y_true - y_pred)
    condition = residual <= delta
    squared_loss = 0.5 * residual**2
    linear_loss = delta * residual - 0.5 * delta**2
    huber = np.where(condition, squared_loss, linear_loss).mean()
    
    return {'rmse': rmse, 'huber': huber}

def huber_score(y_true, y_pred, delta=1.0):
    """최적화용 Huber Score (작을수록 좋음)"""
    return evaluate_predictions(y_true, y_pred, delta)['huber']

print("Huber Loss 함수 정의 완료")

Huber Loss 함수 정의 완료


In [ ]:
# 데이터 파일 로드
if IN_COLAB:
    print("파일 업로드 방법 선택:")
    print("1. 직접 업로드")
    print("2. Google Drive")

    method = input("선택 (1 또는 2): ")

    if method == "1":
        uploaded = files.upload()
        files_list = list(uploaded.keys())
        train_path = [f for f in files_list if 'train' in f.lower()][0]
        test_path = [f for f in files_list if 'test' in f.lower()][0]
    else:
        drive.mount('/content/drive')
        train_path = "/content/drive/MyDrive/train_heat.csv"
        test_path = "/content/drive/MyDrive/test_heat.csv"
else:
    train_path = '/dataset/train_heating.csv' ## 경로 수정 필요
    test_path = '/dataset/test_heating.csv' ## 경로 수정 필요

print(f"파일 경로 설정 완료")

파일 경로 설정 완료


## 1. 고도화된 데이터 (결측치 플래그 생성)

In [38]:
def load_and_preprocess_advanced(train_path, test_path):
    print("고도화된 데이터 로드 및 전처리...")

    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    def process_df_advanced(df):
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
        df.columns = [col.replace('train_heat.', '') for col in df.columns]

        if df['tm'].dtype == 'object':
            df['tm'] = pd.to_datetime(df['tm'])
        else:
            df['tm'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
        
        df['year'] = df['tm'].dt.year
        df['month'] = df['tm'].dt.month
        df['day'] = df['tm'].dt.day
        df['hour'] = df['tm'].dt.hour
        df['dayofweek'] = df['tm'].dt.dayofweek
        df['dayofyear'] = df['tm'].dt.dayofyear

        # ✅ wd (풍향) 제외, 사용할 컬럼만 포함
        missing_cols = ['ta', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand'] # 열수요도 결측치 있음

        # ✅ 1단계: 결측치 플래그 생성 (NaN 변환 전에)
        print("   결측치 플래그 생성 중...")
        for col in missing_cols:
            if col in df.columns:
                # -99를 결측치로 인식하여 플래그 생성
                missing_mask = (df[col] == -99)
                df[f'{col}_missing'] = missing_mask.astype(int)
        
        # ✅ 2단계: 결측치를 NaN으로 변환
        for col in missing_cols:
            if col in df.columns:
                df[col] = df[col].replace(-99, np.nan)

        # ✅ wd 컬럼 처리: -9.9 값을 NaN으로 변환 # SVR 보간에 활용
        if 'wd' in df.columns:
            df['wd'] = df['wd'].replace(-9.9, np.nan)
            print("   wd (풍향) 컬럼의 -9.9 값을 NaN으로 변환됨")
            
        # 일사량 야간은 0 처리
        if 'si' in df.columns:
            night_mask = (df['hour'] < 8) | (df['hour'] > 18)
            df.loc[night_mask & df['si'].isna(), 'si'] = 0

        df = df.sort_values(['branch_id', 'tm'])
        
        # ✅ 3단계: 생성된 결측치 플래그 확인
        missing_flag_cols = [col for col in df.columns if col.endswith('_missing')]
        print(f"   생성된 결측치 플래그: {missing_flag_cols}")
        for col in missing_flag_cols:
            missing_count = df[col].sum()
            print(f"     {col}: {missing_count:,}개 결측치")

        return df

    train_df = process_df_advanced(train_df)
    test_df = process_df_advanced(test_df)

    print(f"   훈련: {train_df.shape}, 테스트: {test_df.shape}")
    print(f"   기간: {train_df['tm'].min()} ~ {test_df['tm'].max()}")

    return train_df, test_df

## 2-1. Feature 생성 : 시즌별 이상치 플래그 생성 (도메인 특화) _ 온도, 풍속, 강수량

In [8]:
def create_weather_outlier_flags(train_df, test_df):
    """시즌별 기상데이터 기반 이상치 플래그 (TRAIN 기준 적용)"""
    print("시즌별 기상 이상치 플래그 생성 중 (TRAIN 기준)...")
    
    # 1단계: TRAIN 데이터에서 시즌별, 지사별 임계값 계산
    outlier_thresholds = {}
    
    for branch in train_df['branch_id'].unique():
        branch_data = train_df[train_df['branch_id'] == branch]
        outlier_thresholds[branch] = {}
        
        # 시즌별로 구분하여 임계값 계산
        for season in [0, 1]:  # 0: 비난방철, 1: 난방철
            season_data = branch_data[branch_data['heating_season'] == season]
            
            if len(season_data) > 10:  # 최소 데이터 요구량
                outlier_thresholds[branch][season] = {
                    # 🌡️ 온도: 하위 10% (극한 추위)
                    'ta_q10': season_data['ta'].quantile(0.10),
                    # 💨 풍속: 상위 10% (강풍)
                    'ws_q90': season_data['ws'].quantile(0.90),
                    # 🌧️ 일강수량: 상위 10% (폭우)
                    'rn_day_q90': season_data['rn_day'].quantile(0.90)
                }
                print(f"   지사 {branch}, {'난방철' if season else '비난방철'}: 임계값 계산 완료")
    
    # 2단계: 임계값을 TRAIN과 TEST에 적용
    def apply_weather_thresholds(df, thresholds):
        df = df.copy()
        # 기본값으로 초기화
        df['cold_extreme'] = 0      # 극한 추위 (하위 10%)
        df['strong_wind'] = 0       # 강풍 (상위 10%)
        df['heavy_rain'] = 0        # 폭우 (상위 10%)
        
        for branch in df['branch_id'].unique():
            if branch in thresholds:
                branch_mask = df['branch_id'] == branch
                
                # 시즌별로 다른 임계값 적용
                for season in [0, 1]:  # 0: 비난방철, 1: 난방철
                    if season in thresholds[branch]:
                        season_mask = branch_mask & (df['heating_season'] == season)
                        season_thresholds = thresholds[branch][season]
                        
                        # 온도 이상치 (낮은 온도)
                        df.loc[season_mask, 'cold_extreme'] = (
                            df.loc[season_mask, 'ta'] < season_thresholds['ta_q10']
                        ).astype(int)
                        
                        # 풍속 이상치 (높은 풍속)
                        df.loc[season_mask, 'strong_wind'] = (
                            df.loc[season_mask, 'ws'] > season_thresholds['ws_q90']
                        ).astype(int)
                        
                        # 강수량 이상치 (많은 비)
                        df.loc[season_mask, 'heavy_rain'] = (
                            df.loc[season_mask, 'rn_day'] > season_thresholds['rn_day_q90']
                        ).astype(int)
                        
        return df
    
    # TRAIN 적용
    train_result = apply_weather_thresholds(train_df, outlier_thresholds)
    
    # TEST 적용
    test_result = apply_weather_thresholds(test_df, outlier_thresholds)
    
    print(f"   기상 이상치 플래그 생성 완료: {len(outlier_thresholds)}개 지사")
    
    return train_result, test_result, outlier_thresholds

## 2-2. Feature 생성 : 시즌 고도화된 특성

In [9]:
def create_advanced_features(df, season_type="heating"):
    df = df.copy()
    print(f"{season_type} 시즌 고도화된 특성 생성 중...")
    
    # 범주형 시간 변수 (문자열로 명시적 변환)
    df['hour_cat'] = df['hour'].astype(str)
    df['month_cat'] = df['month'].astype(str)
    df['weekday_name'] = df['dayofweek'].map(
        lambda x: ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][x]
    ).astype(str)
    
    # 순환 인코딩
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    # 시즌별 월 순환 (sin, cos 포함)
    if season_type == "heating":
        heating_months = {10:0, 11:1, 12:2, 1:3, 2:4, 3:5, 4:6}
        df['heating_month_order'] = df['month'].map(heating_months)
        df['heating_month_sin'] = np.sin(2 * np.pi * df['heating_month_order'] / 7)
        df['heating_month_cos'] = np.cos(2 * np.pi * df['heating_month_order'] / 7)
    else:
        non_heating_months = {5:0, 6:1, 7:2, 8:3, 9:4}
        df['non_heating_month_order'] = df['month'].map(non_heating_months)
        df['non_heating_month_sin'] = np.sin(2 * np.pi * df['non_heating_month_order'] / 5)
        df['non_heating_month_cos'] = np.cos(2 * np.pi * df['non_heating_month_order'] / 5)
    
    # 브랜치 ID (문자열로 변환)
    df['branch_id'] = df['branch_id'].astype(str)
    
    # 고급 기상 범주형 변수
    df['temp_category'] = 'Normal'
    df.loc[df['ta'] < -10, 'temp_category'] = 'VeryCold'
    df.loc[(df['ta'] >= -10) & (df['ta'] < 0), 'temp_category'] = 'Cold'
    df.loc[(df['ta'] >= 0) & (df['ta'] < 10), 'temp_category'] = 'Cool'
    df.loc[(df['ta'] >= 10) & (df['ta'] < 25), 'temp_category'] = 'Normal'
    df.loc[df['ta'] >= 25, 'temp_category'] = 'Hot'
    df['temp_category'] = df['temp_category'].astype(str)
    
    if season_type == "heating":
        df['cold_warning_level'] = 'Normal'
        df.loc[df['ta'] <= -12, 'cold_warning_level'] = 'ColdAdvisory'
        df.loc[df['ta'] <= -15, 'cold_warning_level'] = 'ColdWarning'
        df['cold_warning_level'] = df['cold_warning_level'].astype(str)
    
    df['wind_category'] = 'Weak'
    df.loc[df['ws'] >= 5.0, 'wind_category'] = 'Moderate'
    df.loc[df['ws'] >= 10.0, 'wind_category'] = 'Strong'
    df['wind_category'] = df['wind_category'].astype(str)
    
    # 공휴일/피크시간
    kr_holidays = holidays.KR()
    df['is_holiday'] = df['tm'].dt.date.apply(lambda x: x in kr_holidays)
    df['holiday_type'] = df['is_holiday'].map({False: 'Weekday', True: 'Holiday'}).astype(str)
    
    df['peak_time'] = 'Normal'
    df.loc[(df['hour'] >= 0) & (df['hour'] <= 6), 'peak_time'] = 'Dawn'
    df.loc[(df['hour'] > 6) & (df['hour'] <= 11), 'peak_time'] = 'Morning'
    df.loc[(df['hour'] > 11) & (df['hour'] <= 18), 'peak_time'] = 'Afternoon'
    df.loc[(df['hour'] > 18) & (df['hour'] <= 23), 'peak_time'] = 'Evening'
    df['peak_time'] = df['peak_time'].astype(str)
    
    # 고급 수치형 특성
    df['HDD18'] = np.maximum(0, 18 - df['ta'])
    # df['HDD20'] = np.maximum(0, 20 - df['ta'])
    
    # apparent_temp는 난방시즌에만 생성
    if season_type == "heating":
        def calculate_apparent_temp(ta, ws):
            winter_at = 13.12 + 0.6215 * ta - 11.37 * (ws * 3.6)**0.16 + 0.3965 * ta * (ws * 3.6)**0.16
            return winter_at
        
        df['apparent_temp'] = calculate_apparent_temp(df['ta'], df['ws'])
        df['apparent_temp'] = df['apparent_temp'].fillna(0)
    
    for lag in [3, 6, 24]:
        df[f'ta_lag_{lag}h'] = df.groupby('branch_id')['ta'].shift(lag).fillna(0) # 초기 비어있는 값은 0으로 반영
    
    for window in [6, 12, 24]:
        df[f'ta_ma_{window}h'] = df.groupby('branch_id')['ta'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
    
    df['ta_diff_3h'] = df.groupby('branch_id')['ta'].diff(3)
    df['ta_diff_6h'] = df.groupby('branch_id')['ta'].diff(6)
    # diff 변수 결측치를 0으로 채우기
    df['ta_diff_3h'] = df['ta_diff_3h'].fillna(0)
    df['ta_diff_6h'] = df['ta_diff_6h'].fillna(0)
    
    daily_stats = df.groupby(['branch_id', df['tm'].dt.date]).agg({
        'ta': ['min', 'max', 'mean']
    }).round(2)
    daily_stats.columns = ['daily_ta_min', 'daily_ta_max', 'daily_ta_mean']
    daily_stats['daily_temp_range'] = daily_stats['daily_ta_max'] - daily_stats['daily_ta_min']
    
    df = df.merge(
        daily_stats.reset_index(),
        left_on=['branch_id', df['tm'].dt.date],
        right_on=['branch_id', 'tm'],
        how='left',
        suffixes=('', '_daily')
    )
    
    print(f"   {season_type} 시즌 고도화된 특성 생성 완료: {df.shape[1]}개 컬럼")
    return df

## 2-3. 결측치 보간 (Branch별 SVR)

In [10]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

def apply_svr_interpolation(df, is_train=True):
    """SVR 기반 고급 보간 함수 (폴백 없음, 실패시 에러)"""
    print(f"SVR 보간 적용 중 ({'TRAIN' if is_train else 'TEST'} 데이터)...")
    
    df_interpolated = df.copy()
    
    # 보간 대상 컬럼 확장 (is_train에 따라 heat_demand 포함 여부 결정)
    base_cols = ['ta', 'hm', 'ws', 'wd', 'rn_day', 'rn_hr1', 'si', 'ta_chi']
    if is_train:
        interpolation_cols = base_cols + ['heat_demand']
    else:
        interpolation_cols = base_cols
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    available_cols = [col for col in interpolation_cols if col in numeric_cols]
    
    print(f"   보간 대상 컬럼: {available_cols}")
    
    # 시간 특성 생성 (SVR용)
    df_interpolated['hour'] = df_interpolated['tm'].dt.hour
    df_interpolated['day_of_year'] = df_interpolated['tm'].dt.dayofyear
    df_interpolated['month'] = df_interpolated['tm'].dt.month
    df_interpolated['dayofweek'] = df_interpolated['tm'].dt.dayofweek
    
    # 브랜치별 SVR 보간
    for branch in tqdm(df['branch_id'].unique(), desc="지사별 SVR 보간"):
        branch_mask = df_interpolated['branch_id'] == branch
        branch_data = df_interpolated[branch_mask].copy().sort_values('tm')
        
        if len(branch_data) < 24:  # 최소 데이터 요구량
            raise ValueError(f"지사 {branch}: 데이터 부족 ({len(branch_data)}개). 최소 24개 필요.")
            
        for col in available_cols:
            if col not in branch_data.columns:
                continue
                
            missing_mask = branch_data[col].isna()
            missing_count = missing_mask.sum()
            total_count = len(branch_data)
            
            if missing_count == 0:  # 결측치가 없으면 스킵
                continue
            
            try:
                # 훈련용 데이터 (결측이 아닌 것들)
                train_mask = ~missing_mask
                
                if train_mask.sum() < 10:  # 최소 10개 이상의 훈련 데이터 필요
                    raise ValueError(f"지사 {branch}, 컬럼 {col}: 훈련 데이터 부족 ({train_mask.sum()}개). 최소 10개 필요.")
                
                # 특성: 시간, 연중일, 월, 요일
                feature_cols = ['hour', 'day_of_year', 'month', 'dayofweek']
                X_train = branch_data.loc[train_mask, feature_cols].values
                y_train = branch_data.loc[train_mask, col].values
                
                # 예측할 데이터
                X_pred = branch_data.loc[missing_mask, feature_cols].values
                
                if len(X_pred) == 0:
                    continue
                
                # 스케일링
                scaler_X = StandardScaler()
                scaler_y = StandardScaler()
                
                X_train_scaled = scaler_X.fit_transform(X_train)
                y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
                
                # SVR 모델 훈련
                svr = SVR(kernel='rbf', C=1.0, gamma='scale', epsilon=0.1)
                svr.fit(X_train_scaled, y_train_scaled)
                
                # 예측
                X_pred_scaled = scaler_X.transform(X_pred)
                y_pred_scaled = svr.predict(X_pred_scaled)
                y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
                
                # 결과 할당
                df_interpolated.loc[branch_mask & missing_mask, col] = y_pred
                
                print(f"     지사 {branch}, {col}: {missing_count}개 결측치 SVR 보간 완료")
                
            except Exception as e:
                print(f"❌ 지사 {branch}, 컬럼 {col} SVR 보간 실패:")
                print(f"   에러: {str(e)}")
                print(f"   결측치: {missing_count}/{total_count}개")
                print(f"   훈련 데이터: {train_mask.sum() if 'train_mask' in locals() else 'N/A'}개")
                raise e  # ✅ 폴백 없음, 에러 발생시키고 중단
    
    # 최종 결측치 확인 및 처리
    for col in available_cols:
        if col in df_interpolated.columns:
            remaining_nan = df_interpolated[col].isna().sum()
            if remaining_nan > 0:
                print(f"⚠️ {col}: SVR 보간 후에도 {remaining_nan}개 결측치 남음")
                # Forward/Backward fill로 최종 처리
                df_interpolated[col] = df_interpolated[col].ffill().bfill()
                
                # 여전히 결측치가 있으면 에러
                final_nan = df_interpolated[col].isna().sum()
                if final_nan > 0:
                    raise ValueError(f"{col}: 모든 보간 방법 실패, {final_nan}개 결측치 남음")
    
    print(f"   ✅ SVR 보간 완료")
    return df_interpolated


## 3-1. 규모별 그룹 분할 및 CV 설정


In [11]:
# 1. heating_season 컬럼 추가 함수
def add_heating_season(df):
    """난방 시즌 컬럼 추가"""
    df = df.copy()
    df['heating_season'] = 0  # 기본값: 비난방
    heating_months = [10, 11, 12, 1, 2, 3, 4]  # 10월~4월: 난방시즌
    df.loc[df['month'].isin(heating_months), 'heating_season'] = 1
    return df

# 2. 시즌별 데이터 분할 함수
def split_by_season_only(df):
    """시즌별로만 분할 (2개 그룹)"""
    groups = {}
    
    for season in [0, 1]:  # 0: 비난방, 1: 난방
        season_name = 'heating' if season == 1 else 'non_heating'
        season_data = df[df['heating_season'] == season].copy()
        groups[season_name] = season_data
                
    return groups

# 3. 연도 기반 CV 분할 함수
def create_year_based_cv_splits(df, group_name=""):
    """연도 기반 3-Fold CV 분할 생성"""
    print(f"{group_name} 그룹 - 연도 기반 3-Fold CV 분할 생성...")
    
    # 연도별 데이터 분포 확인
    year_counts = df['year'].value_counts().sort_index()
    print(f"   연도별 데이터 분포:")
    for year, count in year_counts.items():
        print(f"     {year}년: {count:,}개")
    
    # 3-Fold CV: 2021, 2022, 2023년 각각 validation으로 사용
    cv_splits = []
    
    for val_year in [2021, 2022, 2023]:
        train_mask = df['year'] != val_year
        val_mask = df['year'] == val_year
        
        train_indices = df[train_mask].index.tolist()
        val_indices = df[val_mask].index.tolist()
        
        cv_splits.append((train_indices, val_indices))
        
        print(f"   Fold {val_year}: 훈련 {len(train_indices):,}개, 검증 {len(val_indices):,}개")
    
    return cv_splits

## 3-2. 데이터셋 전처리 및 특성 생성

In [21]:
# 1. 기본 전처리 (공통)
print("1️⃣ 기본 전처리...")
train_df, test_df = load_and_preprocess_advanced(train_path, test_path)

# 2. heating_season 컬럼 추가
print("2️⃣ heating_season 컬럼 추가...")
train_df = add_heating_season(train_df)
test_df = add_heating_season(test_df)

# 3. ✅ 결측치 보간 먼저! (파생변수 생성 전)
print("3️⃣ 결측치 SVR 보간 적용...")
print("\n🔧 훈련 데이터 SVR 보간:")
train_df = apply_svr_interpolation(train_df, is_train=True)

print("\n🔧 테스트 데이터 SVR 보간:")
test_df = apply_svr_interpolation(test_df, is_train=False)

# 4. 이상치 플래그 생성 (보간 후)
print("4️⃣ 이상치 플래그 생성...")
train_df, test_df, weather_thresholds = create_weather_outlier_flags(train_df, test_df)

# 5. 그룹별로 먼저 분할
print("5️⃣ 시즌별 그룹 분할...")
train_groups = split_by_season_only(train_df)
test_groups = split_by_season_only(test_df)

# 6. 각 그룹별로 시즌에 맞는 특성 생성 (보간 완료된 데이터로)
print("6️⃣ 그룹별 고도화된 특성 생성...")

# heating 그룹 특성 생성
print("  🔥 heating 그룹 특성 생성 중...")
train_groups['heating'] = create_advanced_features(train_groups['heating'], "heating")
test_groups['heating'] = create_advanced_features(test_groups['heating'], "heating")

# non_heating 그룹 특성 생성  
print("  ❄️ non_heating 그룹 특성 생성 중...")
train_groups['non_heating'] = create_advanced_features(train_groups['non_heating'], "non_heating")
test_groups['non_heating'] = create_advanced_features(test_groups['non_heating'], "non_heating")

print(f"\n📊 그룹별 처리 후 데이터 크기:")
for group_name in ['heating', 'non_heating']:
    print(f"   {group_name:12s}: 훈련 {train_groups[group_name].shape}, 테스트 {test_groups[group_name].shape}")

print(f"\n✅ 모든 전처리 완료!")

# 7. 그룹별 CV 분할 생성 및 결과 확인
print("7️⃣ 그룹별 CV 분할 미리보기:")
for group_name, group_data in train_groups.items():
    if len(group_data) > 100:
        cv_splits = create_year_based_cv_splits(group_data, group_name)
        print(f"   {group_name}: {len(cv_splits)}개 fold 생성됨")
        
        # 결측치 최종 확인
        weather_cols = ['ta', 'hm', 'ws', 'rn_day', 'rn_hr1', 'si', 'ta_chi', 'apparent_temp']
        total_missing = 0
        for col in weather_cols:
            if col in group_data.columns:
                missing = group_data[col].isna().sum()
                total_missing += missing
                if missing > 0:
                    print(f"     ⚠️ {col}: {missing}개 결측치 남음")
        
        if total_missing == 0:
            print(f"     ✅ {group_name}: 모든 기상변수 결측치 해결됨")
        print()

1️⃣ 기본 전처리...
고도화된 데이터 로드 및 전처리...
   결측치 플래그 생성 중...
   wd (풍향) 컬럼의 -9.9 값을 NaN으로 변환됨
   생성된 결측치 플래그: ['ta_missing', 'ws_missing', 'rn_day_missing', 'rn_hr1_missing', 'hm_missing', 'si_missing', 'ta_chi_missing', 'heat_demand_missing']
     ta_missing: 0개 결측치
     ws_missing: 0개 결측치
     rn_day_missing: 0개 결측치
     rn_hr1_missing: 0개 결측치
     hm_missing: 0개 결측치
     si_missing: 0개 결측치
     ta_chi_missing: 0개 결측치
     heat_demand_missing: 0개 결측치
   결측치 플래그 생성 중...
   wd (풍향) 컬럼의 -9.9 값을 NaN으로 변환됨
   생성된 결측치 플래그: ['ta_missing', 'ws_missing', 'rn_day_missing', 'rn_hr1_missing', 'hm_missing', 'si_missing', 'ta_chi_missing', 'heat_demand_missing']
     ta_missing: 0개 결측치
     ws_missing: 0개 결측치
     rn_day_missing: 0개 결측치
     rn_hr1_missing: 0개 결측치
     hm_missing: 0개 결측치
     si_missing: 0개 결측치
     ta_chi_missing: 0개 결측치
     heat_demand_missing: 0개 결측치
   훈련: (289997, 63), 테스트: (97147, 63)
   기간: 2021-01-01 01:00:00 ~ 2025-01-01 00:00:00
2️⃣ heating_season 컬럼 추가...
3️⃣ 결측치 SVR 보간 적용...

지사별 SVR 보간:   0%|          | 0/19 [00:00<?, ?it/s]

   ✅ SVR 보간 완료

🔧 테스트 데이터 SVR 보간:
SVR 보간 적용 중 (TEST 데이터)...
   보간 대상 컬럼: ['ta', 'hm', 'ws', 'wd', 'rn_day', 'rn_hr1', 'si', 'ta_chi']


지사별 SVR 보간:   0%|          | 0/19 [00:00<?, ?it/s]

   ✅ SVR 보간 완료
4️⃣ 이상치 플래그 생성...
시즌별 기상 이상치 플래그 생성 중 (TRAIN 기준)...
   지사 A, 난방철: 임계값 계산 완료
   지사 B, 난방철: 임계값 계산 완료
   지사 C, 난방철: 임계값 계산 완료
   지사 D, 난방철: 임계값 계산 완료
   지사 E, 난방철: 임계값 계산 완료
   지사 F, 난방철: 임계값 계산 완료
   지사 G, 난방철: 임계값 계산 완료
   지사 H, 난방철: 임계값 계산 완료
   지사 I, 난방철: 임계값 계산 완료
   지사 J, 난방철: 임계값 계산 완료
   지사 K, 난방철: 임계값 계산 완료
   지사 L, 난방철: 임계값 계산 완료
   지사 M, 난방철: 임계값 계산 완료
   지사 N, 난방철: 임계값 계산 완료
   지사 O, 난방철: 임계값 계산 완료
   지사 P, 난방철: 임계값 계산 완료
   지사 Q, 난방철: 임계값 계산 완료
   지사 R, 난방철: 임계값 계산 완료
   지사 S, 난방철: 임계값 계산 완료
   기상 이상치 플래그 생성 완료: 19개 지사
5️⃣ 시즌별 그룹 분할...
6️⃣ 그룹별 고도화된 특성 생성...
  🔥 heating 그룹 특성 생성 중...
heating 시즌 고도화된 특성 생성 중...
   heating 시즌 고도화된 특성 생성 완료: 68개 컬럼
heating 시즌 고도화된 특성 생성 중...
   heating 시즌 고도화된 특성 생성 완료: 68개 컬럼
  ❄️ non_heating 그룹 특성 생성 중...
non_heating 시즌 고도화된 특성 생성 중...
   non_heating 시즌 고도화된 특성 생성 완료: 71개 컬럼
non_heating 시즌 고도화된 특성 생성 중...
   non_heating 시즌 고도화된 특성 생성 완료: 71개 컬럼

📊 그룹별 처리 후 데이터 크기:
   heating     : 훈련 (289997, 68), 테스트 (97147, 68)
   non_heating 

### 전처리 데이터 저장

In [ ]:
def save_processed_data(train_groups, test_groups, weather_thresholds):
   """전처리된 데이터를 저장"""
   os.makedirs(save_dir, exist_ok=True)
   
   # 각 그룹별로 CSV 저장
   for group_name in train_groups.keys():
       train_groups[group_name].to_csv(f"/train_{group_name}.csv", index=False)
       test_groups[group_name].to_csv(f"/test_{group_name}.csv", index=False)
   
   # weather_thresholds 저장
   with open(f"/weather_thresholds.pickle", 'wb') as f:
       pickle.dump(weather_thresholds, f)
   
   print(f"✅ 전처리된 데이터가 폴더에 저장되었습니다.")

save_processed_data(train_groups, test_groups, weather_thresholds)

## ! 데이터 있을 때, 여기부터 바로 돌리면 됨

In [26]:
def load_processed_data(save_dir="/Users/jisupark_1/workspace/star_track_python/PRJ_Meteo/dataset/"):
   """저장된 전처리 데이터를 로드"""
   try:
       train_groups = {
           'heating': pd.read_csv(f"{save_dir}/train_heating.csv"),
           'non_heating': pd.read_csv(f"{save_dir}/train_non_heating.csv")
       }
       
       test_groups = {
           'heating': pd.read_csv(f"{save_dir}/test_heating.csv"),
           'non_heating': pd.read_csv(f"{save_dir}/test_non_heating.csv")
       }
       
       with open(f"{save_dir}/weather_thresholds.pickle", 'rb') as f:
           weather_thresholds = pickle.load(f)
       
       print(f"✅ 저장된 데이터를 로드했습니다.")
       return train_groups, test_groups, weather_thresholds
   
   except FileNotFoundError:
       print("❌ 저장된 데이터가 없습니다.")
       return None, None, None

# 1. 저장된 데이터 로드 시도
train_groups, test_groups, weather_thresholds = load_processed_data()

✅ 저장된 데이터를 로드했습니다.


In [27]:
print("7️⃣ 그룹별 CV 분할 미리보기:")
for group_name, group_data in train_groups.items():
    if len(group_data) > 100:
        cv_splits = create_year_based_cv_splits(group_data, group_name)
        print(f"   {group_name}: {len(cv_splits)}개 fold 생성됨")
        
        # 결측치 최종 확인
        weather_cols = [# 기본 시간 변수 추가
            'hour', 'month', 'day',
            # 기상 변수
            'ta', 'hm', 'ws', 'rn_day', 'rn_hr1', 'si', 'ta_chi',  # rn_hr1, ta_chi 추가
            # 파생 변수
            'HDD18', 'apparent_temp',  # HDD20 제거
            # 순환 인코딩
            'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 
            'dayofweek_sin', 'dayofweek_cos',
            # 시계열 특성
            'ta_lag_3h', 'ta_lag_6h', 'ta_lag_24h', 
            'ta_ma_6h', 'ta_ma_12h', 'ta_ma_24h',
            'ta_diff_3h', 'ta_diff_6h', 
            # 일별 통계
            'daily_ta_min', 'daily_ta_max', 'daily_ta_mean', 'daily_temp_range']
        total_missing = 0
        for col in weather_cols:
            if col in group_data.columns:
                missing = group_data[col].isna().sum()
                total_missing += missing
                if missing > 0:
                    print(f"     ⚠️ {col}: {missing}개 결측치 남음")
        
        if total_missing == 0:
            print(f"     ✅ {group_name}: 모든 기상변수 결측치 해결됨")
        print()

7️⃣ 그룹별 CV 분할 미리보기:
heating 그룹 - 연도 기반 3-Fold CV 분할 생성...
   연도별 데이터 분포:
     2021년: 96,653개
     2022년: 96,672개
     2023년: 96,672개
   Fold 2021: 훈련 193,344개, 검증 96,653개
   Fold 2022: 훈련 193,325개, 검증 96,672개
   Fold 2023: 훈련 193,325개, 검증 96,672개
   heating: 3개 fold 생성됨
     ✅ heating: 모든 기상변수 결측치 해결됨

non_heating 그룹 - 연도 기반 3-Fold CV 분할 생성...
   연도별 데이터 분포:
     2021년: 69,768개
     2022년: 69,768개
     2023년: 69,768개
   Fold 2021: 훈련 139,536개, 검증 69,768개
   Fold 2022: 훈련 139,536개, 검증 69,768개
   Fold 2023: 훈련 139,536개, 검증 69,768개
   non_heating: 3개 fold 생성됨
     ✅ non_heating: 모든 기상변수 결측치 해결됨



## 4. 모델별 피쳐 정의

Prophet

- 시간 변수와 기본 기상 변수 중심
- 너무 많은 변수보다는 핵심 특성에 집중

CatBoost

- 범주형 변수와 플래그 변수를 최대한 활용
- 결측치/이상치 플래그로 데이터 품질 정보 제공

In [ ]:
def define_model_features():
    # Prophet은 자체 시계열 분해 능력이 있음
    prophet_features = {
        'basic': [
            'ta', 'hm', 'ws', 'HDD18', 'apparent_temp'
        ],
        'seasonal': [
            'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 
            'dayofweek_sin', 'dayofweek_cos',
            'ta_lag_3h', 'ta_lag_6h'  # 짧은 lag만 (Prophet 자체 시계열 처리 보완) # ma, diff는 불필요 (Prophet이 자체 처리)
        ]
    }
    # CatBoost: 모든 특성 활용 (범주형 + 시계열 + 플래그)
    catboost_features = {
        'numerical': [
            'day', 'dayofyear',
            # 기상 변수
            'ta', 'hm', 'ws', 'rn_day', 'rn_hr1', 'si', 'ta_chi',
            # 파생 변수
            'HDD18',
            # 순환 인코딩
            'hour_sin', 'hour_cos', #'month_sin', 'month_cos' 제외 (중복)
            'dayofweek_sin', 'dayofweek_cos',
            # 모든 시계열 특성 (CatBoost는 직접 학습)
            'ta_lag_3h', 'ta_lag_6h', 'ta_lag_24h', 
            'ta_ma_6h', 'ta_ma_12h', 'ta_ma_24h',
            'ta_diff_3h', 'ta_diff_6h', 
            # 일별 통계
            'daily_ta_min', 'daily_ta_max', 'daily_ta_mean', 'daily_temp_range'
        ],
        'categorical': [
            'branch_id', 'hour_cat', 'month_cat', 'weekday_name', 
            'temp_category', 'wind_category', 'holiday_type', 'peak_time'
        ],
        'flags': [
            # 결측치 플래그 (모든 기상 변수)
            'ta_missing', 'ws_missing', 'rn_day_missing', 
            'rn_hr1_missing', 'hm_missing', 'si_missing', 'ta_chi_missing',
            # 이상치 플래그 
            'cold_extreme', 'strong_wind', 'heavy_rain'
        ]
    }
     # LSTM: 핵심 특성 + 전처리된 시계열 (lag 제외)
    lstm_features = {
        'numerical': [
            # 기본 시간 변수 추가
            'day', 'dayofyear',
            # 핵심 기상 변수
            'ta', 'hm', 'ws', 'rn_day', 'si',
            # 파생 변수
            'HDD18',
            # 순환 인코딩
            'hour_sin', 'hour_cos', #'month_sin', 'month_cos' 제외 (중복)
            'dayofweek_sin', 'dayofweek_cos',
            # 핵심 이동 평균만
            'ta_ma_6h', 'ta_ma_12h', 'ta_ma_24h',
            # 일별 통계 (핵심만)
            'daily_ta_mean', 'daily_temp_range' # lag, diff 제거 - LSTM sequence로 대체
        ], 
        'categorical_encoded': ['branch_id']
    }
    
    # heating 시즌별 특성 추가
    prophet_heating = copy.deepcopy(prophet_features)
    prophet_heating['basic'].append('apparent_temp')
    prophet_heating['seasonal'].extend(['heating_month_order'])
     # prophet_non_heating도 non_heating_month_order 추가
    prophet_non_heating = copy.deepcopy(prophet_features)
    prophet_non_heating['seasonal'].append('non_heating_month_order')

    # 난방시즌용 CatBoost (한파 경보 + 시즌별 순환 추가)
    catboost_heating = copy.deepcopy(catboost_features)
    catboost_heating['categorical'].append('cold_warning_level')
    catboost_heating['seasonal'] = [
        'apparent_temp', 'heating_month_order', 
        'heating_month_sin', 'heating_month_cos'
    ]
    # 비난방시즌용 CatBoost (시즌별 순환 추가)
    catboost_non_heating = copy.deepcopy(catboost_features)
    catboost_non_heating['seasonal'] = [
        'non_heating_month_order', 'non_heating_month_sin', 'non_heating_month_cos'
    ]

    
    return {
        'prophet_heating': prophet_heating,
        'prophet_non_heating': prophet_non_heating,  # apparent_temp 없음
        'catboost_heating': catboost_heating,
        'catboost_non_heating': catboost_non_heating
        # 'lstm_heating': lstm_heating,
        # 'lstm_non_heating': lstm_non_heating
    }

model_features = define_model_features()

print("모델별 피쳐 정의 완료:")
print("=" * 50)
for model_name, features in model_features.items():
    total_features = sum(len(v) if isinstance(v, list) else 0 for v in features.values())
    print(f"{model_name:20s}: {total_features}개 피쳐")
    for feature_type, feature_list in features.items():
        if isinstance(feature_list, list):
            print(f"  {feature_type:15s}: {len(feature_list)}개")

모델별 피쳐 정의 완료:
prophet_heating     : 15개 피쳐
  basic          : 6개
  seasonal       : 9개
prophet_non_heating : 14개 피쳐
  basic          : 5개
  seasonal       : 9개
catboost_heating    : 49개 피쳐
  numerical      : 26개
  categorical    : 9개
  flags          : 10개
  seasonal       : 4개
catboost_non_heating: 47개 피쳐
  numerical      : 26개
  categorical    : 8개
  flags          : 10개
  seasonal       : 3개


## 5. 모델 클래스들 (Huber Loss 지원)

In [ ]:

# Time Series Dataset 클래스 추가
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, sequence_length=24):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.X) - self.sequence_length + 1
    
    def __getitem__(self, idx):
        return (
            self.X[idx:idx+self.sequence_length],
            self.y[idx+self.sequence_length-1]
        )

## 6. Prophet, CatBoost, LSTM 모델 클래스 (Optuna 최적화)

In [15]:
class ProphetOptimizedModel:
    def __init__(self, season_type="heating"):
        self.models = {}
        self.best_params = {}
        self.season_type = season_type
        
        
    def optimize_hyperparameters(self, df, cv_splits, target_col='heat_demand', n_trials=30):
        """연도 기반 CV를 사용한 하이퍼파라미터 최적화"""
        print(f"Prophet Huber Loss 하이퍼파라미터 최적화 중... (trials: {n_trials})")
        
        def objective(trial):
            # 하이퍼파라미터 샘플링
            changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
            seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.1, 10, log=True)
            holidays_prior_scale = trial.suggest_float('holidays_prior_scale', 0.1, 10, log=True)
            seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
            
            cv_scores = []
            
            # 연도 기반 3-Fold CV
            for fold, (train_idx, val_idx) in enumerate(cv_splits):
                fold_predictions = []
                fold_targets = []
                
                train_fold = df.iloc[train_idx]
                val_fold = df.iloc[val_idx]
                
                # 지사별 모델 훈련 및 예측
                for branch in df['branch_id'].unique():
                    branch_train = train_fold[train_fold['branch_id'] == branch]
                    branch_val = val_fold[val_fold['branch_id'] == branch]
                    
                    if len(branch_train) < 50 or len(branch_val) == 0:
                        continue
                    
                    try:
                        # Prophet 데이터 준비 (이미 보간된 데이터 사용)
                        prophet_df = pd.DataFrame({
                            'ds': pd.to_datetime(branch_train['tm']),
                            'y': branch_train[target_col]
                        })
                        
                        # 시즌별 피쳐 설정 사용
                        feature_config = model_features[f'prophet_{self.season_type}']
                        regressors = feature_config['basic'] + feature_config['seasonal']
                        
                        for reg in regressors:
                            if reg in branch_train.columns:
                                prophet_df[reg] = branch_train[reg].values
                        
                        # Prophet 모델 생성 및 훈련
                        model = Prophet(
                            changepoint_prior_scale=changepoint_prior_scale,
                            seasonality_prior_scale=seasonality_prior_scale,
                            holidays_prior_scale=holidays_prior_scale,
                            seasonality_mode=seasonality_mode,
                            daily_seasonality=True,
                            weekly_seasonality=True,
                            yearly_seasonality=True
                        )
                        
                        # 회귀변수 추가
                        for reg in regressors:
                            if reg in prophet_df.columns and reg not in ['ds', 'y']:
                                model.add_regressor(reg)
                        
                        model.fit(prophet_df)
                        
                        # 예측 데이터 준비
                        future_df = pd.DataFrame({
                            'ds': pd.to_datetime(branch_val['tm'])
                        })
                        
                        for reg in regressors:
                            if reg in branch_val.columns:
                                future_df[reg] = branch_val[reg].values
                        
                        # 예측 실행
                        forecast = model.predict(future_df)
                        predictions = np.maximum(forecast['yhat'].values, 0)
                        
                        actual_values = branch_val[target_col].values
                        valid_mask = ~np.isnan(actual_values) & ~np.isnan(predictions)
                        
                        if valid_mask.sum() > 0:
                            fold_predictions.extend(predictions[valid_mask])
                            fold_targets.extend(actual_values[valid_mask])
                    
                    except Exception as e:
                        print(f"❌ 최적화 중 지사 {branch} Fold {fold+1} 실패:")
                        print(f"   에러: {str(e)}")
                        raise e
                
                # Fold별 Huber Loss 계산
                # 수정된 코드
                if len(fold_predictions) > 10:
                    # ✅ list를 numpy array로 변환
                    fold_targets_array = np.array(fold_targets)
                    fold_predictions_array = np.array(fold_predictions)
                    
                    huber = huber_score(fold_targets_array, fold_predictions_array, delta=1.0)
                    cv_scores.append(huber)
                    print(f"   Fold {fold+1}: Huber Loss = {huber:.4f} ({len(fold_predictions)}개 예측)")
                else:
                    print(f"   Fold {fold+1}: 유효한 예측 부족 ({len(fold_predictions)}개)")
            
            if len(cv_scores) == 0:
                raise RuntimeError("모든 Fold에서 Prophet 최적화 실패")
                
            return np.mean(cv_scores)
        
        # Optuna 최적화 실행
        study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        self.best_params = study.best_params
        print(f"   Prophet 최적 Huber Loss: {study.best_value:.4f}")
        print(f"   최적 파라미터: {self.best_params}")
        return study.best_value

    def fit(self, df, target_col='heat_demand'):
        """최적화된 파라미터로 전체 데이터 훈련"""
        print(f"Prophet 모델 훈련 중...")
        
        branches = df['branch_id'].unique()
        success_count = 0
        
        # 시즌별 피쳐 설정 사용
        feature_config = model_features[f'prophet_{self.season_type}']
        regressors = feature_config['basic'] + feature_config['seasonal']
        
        print(f"   사용할 regressors: {regressors}")

        for branch in tqdm(branches, desc="Prophet 지사별 훈련"):
            branch_data = df[df['branch_id'] == branch].copy()

            if len(branch_data) < 50:
                print(f"⚠️ 지사 {branch}: 데이터 부족 ({len(branch_data)}개) - 스킵")
                continue

            try:
                # Prophet 데이터 준비
                prophet_df = pd.DataFrame({
                    'ds': branch_data['tm'],
                    'y': branch_data[target_col]
                })

                # 회귀변수 추가
                missing_regressors = []
                for reg in regressors:
                    if reg in branch_data.columns:
                        prophet_df[reg] = branch_data[reg].values
                    else:
                        missing_regressors.append(reg)
                
                if missing_regressors:
                    print(f"⚠️ 지사 {branch}: 누락된 regressors: {missing_regressors}")

                # 최적화된 파라미터로 모델 생성
                model = Prophet(
                    changepoint_prior_scale=self.best_params.get('changepoint_prior_scale', 0.05),
                    seasonality_prior_scale=self.best_params.get('seasonality_prior_scale', 10.0),
                    holidays_prior_scale=self.best_params.get('holidays_prior_scale', 10.0),
                    seasonality_mode=self.best_params.get('seasonality_mode', 'multiplicative'),
                    daily_seasonality=True,
                    weekly_seasonality=True,
                    yearly_seasonality=True  # ✅ 연간 계절성 활성화
                )

                # 회귀변수 추가
                added_regressors = []
                for reg in regressors:
                    if reg in prophet_df.columns and reg not in ['ds', 'y']:
                        model.add_regressor(reg)
                        added_regressors.append(reg)

                # 데이터 품질 확인
                if prophet_df['y'].isna().sum() > 0:
                    print(f"⚠️ 지사 {branch}: 타겟 변수에 결측치 {prophet_df['y'].isna().sum()}개")
                
                model.fit(prophet_df)
                self.models[branch] = model
                success_count += 1

            except Exception as e:
                print(f"❌ 지사 {branch} Prophet 훈련 실패:")
                print(f"   에러: {str(e)}")
                print(f"   데이터 크기: {len(branch_data)}")
                print(f"   Prophet 데이터프레임 크기: {prophet_df.shape}")
                print(f"   추가된 regressors: {added_regressors}")
                print(f"   타겟 변수 통계:")
                print(f"     평균: {prophet_df['y'].mean():.2f}")
                print(f"     결측치: {prophet_df['y'].isna().sum()}개")
                print(f"     최소값: {prophet_df['y'].min():.2f}")
                print(f"     최대값: {prophet_df['y'].max():.2f}")
                
                # 회귀변수별 결측치 확인
                print(f"   회귀변수 결측치 현황:")
                for reg in regressors:
                    if reg in prophet_df.columns:
                        missing = prophet_df[reg].isna().sum()
                        print(f"     {reg}: {missing}개")
                
                raise e  # ✅ 에러 발생시키고 중단

        print(f"   {success_count}/{len(branches)}개 지사 훈련 완료")
        
        if success_count == 0:
            raise RuntimeError("모든 지사에서 Prophet 훈련 실패!")

    def predict(self, df):
        """예측 실행"""
        if len(self.models) == 0:
            raise RuntimeError("훈련된 Prophet 모델이 없습니다!")
        
        predictions = []
        
        # 시즌별 피쳐 설정 사용
        feature_config = model_features[f'prophet_{self.season_type}']
        regressors = feature_config['basic'] + feature_config['seasonal']
        
        for branch in df['branch_id'].unique():
            if branch not in self.models:
                print(f"⚠️ 지사 {branch}: 훈련된 모델 없음, 0으로 채움")
                predictions.extend([0] * len(df[df['branch_id'] == branch]))
                continue

            branch_data = df[df['branch_id'] == branch].copy()
            
            try:
                future_df = pd.DataFrame({'ds': branch_data['tm']})

                # 회귀변수 추가
                for reg in regressors:
                    if reg in branch_data.columns:
                        future_df[reg] = branch_data[reg].values

                forecast = self.models[branch].predict(future_df)
                branch_predictions = np.maximum(forecast['yhat'].values, 0)
                predictions.extend(branch_predictions)
                
            except Exception as e:
                print(f"❌ 지사 {branch} Prophet 예측 실패:")
                print(f"   에러: {str(e)}")
                raise e

        return np.array(predictions)

print("Prophet 최적화 모델 클래스 정의 완료")

Prophet 최적화 모델 클래스 정의 완료


In [16]:
class CatBoostOptimizedModel:
    def __init__(self, season_type="heating"):
        self.model = None
        self.feature_cols = None
        self.categorical_features = None
        self.best_params = {}
        self.season_type = season_type
        
        # 시즌별 피쳐 설정
        feature_config = model_features[f'catboost_{season_type}']
        self.features = feature_config
        
    def prepare_features(self, df):
        """피쳐 준비 및 전처리"""
        df = df.copy()
        
        # 모든 피쳐 수집 (seasonal 추가)
        all_features = []
        for ftype in ['numerical', 'categorical', 'flags', 'seasonal']:
            if ftype in self.features:
                all_features.extend(self.features[ftype])
        
        # 사용 가능한 피쳐만 선택
        available_features = [col for col in all_features if col in df.columns]
        missing_features = [col for col in all_features if col not in df.columns]
        
        if missing_features:
            print(f"⚠️ 누락된 피쳐 ({len(missing_features)}개): {missing_features}")
        
        self.feature_cols = available_features
        
        # 범주형 피쳐 처리
        categorical_features = []
        if 'categorical' in self.features:
            categorical_features = [col for col in self.features['categorical'] if col in df.columns]
            
        self.categorical_features = categorical_features
        
        # 범주형 변수를 문자열로 변환
        for col in categorical_features:
            if col in df.columns:
                df[col] = df[col].astype(str)
        
        print(f"   최종 사용 피쳐: {len(self.feature_cols)}개")
        print(f"   범주형 피쳐: {len(categorical_features)}개 - {categorical_features}")
        
        return df[self.feature_cols], categorical_features
    
    # 단조성 제약 설정 함수 추가
    def _get_monotone_constraints(self, feature_names):
        """난방 수요 예측에 맞는 단조성 제약 설정"""
        constraints = []
        
        for feature in feature_names:
            # ✅ 2. 단조성 제약 설정 (물리적 상식 반영)
            if ('ta' in feature or 'apparent_temp' in feature) and 'lag' not in feature and 'diff' not in feature:  
                # 온도, 체감온도: 온도 ↓ = 난방 수요 ↑ (음의 상관)
                constraints.append(-1)
            elif feature in ['HDD18']:  
                # 난방도일: HDD ↑ = 난방 수요 ↑ (양의 상관)
                constraints.append(1)
            else:  
                # 나머지는 제약 없음 (범주형 변수 cold_extreme 포함)
                constraints.append(0)
        
        constrained_count = sum(1 for c in constraints if c != 0)
        print(f"   단조성 제약: {constrained_count}개 피쳐에 적용")
        return constraints

    def optimize_hyperparameters(self, df, cv_splits, target_col='heat_demand', n_trials=50):
        """연도 기반 CV를 사용한 하이퍼파라미터 최적화"""
        print(f"CatBoost Huber Loss 하이퍼파라미터 최적화 중... (trials: {n_trials})")
        
        # 피쳐 준비
        X_full, categorical_features = self.prepare_features(df)
        y_full = df[target_col].values
        
        print(f"   최적화 데이터: {X_full.shape}")
        print(f"   타겟 통계: 평균={y_full.mean():.2f}, 표준편차={y_full.std():.2f}")
        
        def objective(trial):
            # 하이퍼파라미터 샘플링
            params = {
                'iterations': trial.suggest_int('iterations', 500, 2000),
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 20),
                'border_count': trial.suggest_int('border_count', 32, 255),
                'random_seed': SEED,
                'task_type': 'CPU',
                'verbose': 0,
                'loss_function': 'Huber:delta=1.0'  # CatBoost 내장 Huber Loss
            }
            
            cv_scores = []
            
            # 연도 기반 3-Fold CV
            for fold, (train_idx, val_idx) in enumerate(cv_splits):
                try:
                    X_train, X_val = X_full.iloc[train_idx], X_full.iloc[val_idx]
                    y_train, y_val = y_full[train_idx], y_full[val_idx]
                    
                    # 데이터 크기 확인
                    if len(X_train) < 10 or len(X_val) < 5:
                        print(f"     Fold {fold+1}: 데이터 부족 (train={len(X_train)}, val={len(X_val)})")
                        continue
                    
                    # CatBoost 모델 생성 및 훈련
                    model = CatBoostRegressor(**params, cat_features=categorical_features)
                    model.fit(X_train, y_train, verbose=0)
                    
                    # 예측 및 평가
                    predictions = model.predict(X_val)
                    predictions = np.maximum(predictions, 0)  # 음수 제거
                    
                    # 평가 지표 계산
                    metrics = evaluate_predictions(y_val, predictions, delta=1.0)
                    print(f"     Fold {fold+1}: RMSE={metrics['rmse']:.4f}, Huber={metrics['huber']:.4f}")
                    
                    cv_scores.append(metrics['huber'])  # 최적화는 Huber Loss 기준
                    
                except Exception as e:
                    print(f"❌ Fold {fold+1} CatBoost 최적화 실패:")
                    print(f"   에러: {str(e)}")
                    print(f"   훈련 데이터: {len(X_train) if 'X_train' in locals() else 'N/A'}")
                    print(f"   검증 데이터: {len(X_val) if 'X_val' in locals() else 'N/A'}")
                    raise e  # ✅ 에러 발생시키고 중단
            
            if len(cv_scores) == 0:
                print(f"   ⚠️ 모든 Fold에서 CatBoost 최적화 실패")
                return 999.0
                
            return np.mean(cv_scores)
        
        # Optuna 최적화 실행
        study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        self.best_params = study.best_params
        print(f"   CatBoost 최적 Huber Loss: {study.best_value:.4f}")
        print(f"   최적 파라미터: {self.best_params}")
        return study.best_value

    def fit(self, df, target_col='heat_demand'):
        """최적화된 파라미터로 전체 데이터 훈련"""
        print(f"CatBoost 모델 훈련 중...")
        
        # 피쳐 준비
        X, categorical_features = self.prepare_features(df)
        y = df[target_col].values
        
        # 데이터 품질 확인
        print(f"   훈련 데이터: {X.shape}")
        print(f"   타겟 통계: 평균={y.mean():.2f}, 표준편차={y.std():.2f}, 범위=[{y.min():.2f}, {y.max():.2f}]")
        
        # 결측치 확인
        missing_info = {}
        for col in X.columns:
            missing_count = X[col].isna().sum()
            if missing_count > 0:
                missing_info[col] = missing_count
        
        if missing_info:
            print(f"   ⚠️ 피쳐별 결측치: {missing_info}")
            # 결측치가 있는 경우 처리
            for col, missing_count in missing_info.items():
                if col in categorical_features:
                    X[col] = X[col].fillna('missing')
                else:
                    X[col] = X[col].fillna(X[col].median())
            print(f"   결측치 처리 완료")

        try:
            # ✅ 단조성 제약 계산 ###############################################################
            monotone_constraints = self._get_monotone_constraints(X.columns)
            # 최적화된 파라미터로 모델 생성
            self.model = CatBoostRegressor(
                iterations=self.best_params.get('iterations', 1000),
                learning_rate=self.best_params.get('learning_rate', 0.1),
                depth=self.best_params.get('depth', 6),
                l2_leaf_reg=self.best_params.get('l2_leaf_reg', 3),
                border_count=self.best_params.get('border_count', 128),
                cat_features=categorical_features,
                random_seed=SEED,
                verbose=False,
                allow_writing_files=False,
                loss_function='Huber:delta=1.0',  # Huber Loss 사용
                # ✅ 단조성 제약 추가 적용 ###############################################################
                monotone_constraints=monotone_constraints
            )

            # 모델 훈련
            self.model.fit(X, y)
            print(f"   CatBoost 훈련 완료")
            
            # 피쳐 중요도 출력 (상위 10개)
            if hasattr(self.model, 'feature_importances_'):
                feature_importance = dict(zip(X.columns, self.model.feature_importances_))
                top_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:10]
                print(f"   상위 10개 중요 피쳐:")
                for i, (feature, importance) in enumerate(top_features, 1):
                    print(f"     {i:2d}. {feature}: {importance:.3f}")
        
        except Exception as e:
            print(f"❌ CatBoost 훈련 실패:")
            print(f"   에러: {str(e)}")
            print(f"   데이터 크기: {X.shape}")
            print(f"   범주형 피쳐: {categorical_features}")
            print(f"   파라미터: {self.best_params}")
            raise e  # ✅ 에러 발생시키고 중단

    def predict(self, df):
        """예측 실행"""
        if self.model is None:
            raise RuntimeError("훈련된 CatBoost 모델이 없습니다!")
            
        try:
            # 피쳐 준비
            X, categorical_features = self.prepare_features(df)
            
            # 결측치 처리 (훈련 시와 동일하게)
            for col in X.columns:
                if X[col].isna().sum() > 0:
                    if col in categorical_features:
                        X[col] = X[col].fillna('missing')
                    else:
                        X[col] = X[col].fillna(X[col].median())
            
            predictions = self.model.predict(X)
            predictions = np.maximum(predictions, 0)  # 음수 제거
            
            print(f"   CatBoost 예측 완료: {len(predictions)}개")
            print(f"   예측 통계: 평균={predictions.mean():.2f}, 범위=[{predictions.min():.2f}, {predictions.max():.2f}]")
            
            return predictions
            
        except Exception as e:
            print(f"❌ CatBoost 예측 실패:")
            print(f"   에러: {str(e)}")
            print(f"   데이터 크기: {df.shape}")
            raise e  # ✅ 에러 발생시키고 중단

print("CatBoost 최적화 모델 클래스 정의 완료")

CatBoost 최적화 모델 클래스 정의 완료


## 7. 스태킹 앙상블 클래스 (Ridge 메타모델 최적화) v1

In [18]:
# from sklearn.linear_model import Ridge

# class AdvancedStackingEnsemble:
#     def __init__(self, season_type="heating", group_name=""):
#         self.season_type = season_type
#         self.group_name = group_name
#         self.models = {
#             'prophet': ProphetOptimizedModel(season_type),
#             'catboost': CatBoostOptimizedModel(season_type)
#             # 'lstm': LSTMOptimizedModel(season_type)
#         }
#         self.meta_model = None
#         self.best_meta_params = {}
#         self.individual_scores = {}

#     def optimize_meta_model(self, level1_features, targets, cv_splits, n_trials=20):
#         """Ridge 메타모델 최적화 (연도 기반 CV)"""
#         print(f"Ridge 메타모델 최적화 중... (trials: {n_trials})")
        
#         def objective(trial):
#             # Ridge 파라미터 샘플링
#             alpha = trial.suggest_float('alpha', 0.01, 100.0, log=True)
            
#             cv_scores = []
            
#             # 연도 기반 3-Fold CV
#             for fold, (train_idx, val_idx) in enumerate(cv_splits):
#                 try:
#                     # 레벨1 피쳐에서 해당 인덱스 선택
#                     train_meta_mask = np.isin(range(len(level1_features)), train_idx)
#                     val_meta_mask = np.isin(range(len(level1_features)), val_idx)
                    
#                     X_train_meta = level1_features[train_meta_mask]
#                     X_val_meta = level1_features[val_meta_mask]
#                     y_train_meta = targets[train_meta_mask]
#                     y_val_meta = targets[val_meta_mask]
                    
#                     if len(X_train_meta) < 5 or len(X_val_meta) < 2:
#                         print(f"     메타 Fold {fold+1}: 데이터 부족")
#                         continue
                    
#                     # Ridge 훈련
#                     model = Ridge(alpha=alpha, random_state=SEED)
#                     model.fit(X_train_meta, y_train_meta)
                    
#                     # 예측 및 평가
#                     pred = model.predict(X_val_meta)
#                     pred = np.maximum(pred, 0)  # 음수 제거
                    
#                     rmse = np.sqrt(mean_squared_error(y_val_meta, pred))
#                     cv_scores.append(rmse)
#                     print(f"     메타 Fold {fold+1}: RMSE = {rmse:.4f}")
                    
#                 except Exception as e:
#                     print(f"❌ 메타 Fold {fold+1} 최적화 실패:")
#                     print(f"   에러: {str(e)}")
#                     raise e  # ✅ 에러 발생시키고 중단
                    
#             if len(cv_scores) == 0:
#                 print(f"   ⚠️ 모든 메타 Fold에서 최적화 실패")
#                 return 999.0
                
#             return np.mean(cv_scores)
        
#         # Optuna 최적화 실행
#         study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
#         study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
#         self.best_meta_params = study.best_params
#         print(f"   Ridge 최적 RMSE: {study.best_value:.4f}")
#         print(f"   최적 파라미터: {self.best_meta_params}")
        
#         return study.best_value

#     def fit(self, train_df, cv_splits, target_col='heat_demand', optimize_trials=None):
#         """스태킹 앙상블 훈련 (연도 기반 CV 사용)"""
#         print(f"\n{self.group_name} 스태킹 앙상블 훈련 시작!")
#         print("=" * 60)
        
#         if len(train_df) < 100:
#             raise RuntimeError(f"데이터가 부족합니다 ({len(train_df)}개). 최소 100개 필요.")
            
#         # 기본 trials 설정
#         if optimize_trials is None:
#             optimize_trials = {'prophet': 30, 'catboost': 50, 'meta': 20}

#         print(f"훈련 데이터: {len(train_df):,}개")
#         print(f"연도 분포: {dict(train_df['year'].value_counts().sort_index())}")

#         # 1단계: 개별 모델 하이퍼파라미터 최적화 및 훈련
#         level1_predictions_dict = {}

#         for name, model in self.models.items():
#             print(f"\n{name.upper()} 최적화 및 훈련...")
#             try:
#                 start_time = datetime.now()
                
#                 # 하이퍼파라미터 최적화 (CV 기반)
#                 best_score = model.optimize_hyperparameters(
#                     train_df, cv_splits, target_col, n_trials=optimize_trials[name]
#                 )
                
#                 # 최적화된 파라미터로 전체 훈련 데이터에 재훈련
#                 model.fit(train_df, target_col)
                
#                 # CV를 통한 레벨1 예측값 생성 (Out-of-Fold 예측)
#                 oof_predictions = np.zeros(len(train_df))
                
#                 for fold, (train_idx, val_idx) in enumerate(cv_splits):
#                     print(f"   OOF Fold {fold+1} 처리 중...")
                    
#                     fold_train = train_df.iloc[train_idx]
#                     fold_val = train_df.iloc[val_idx]
                    
#                     # 폴드별 모델 훈련 (최적 파라미터 사용)
#                     if name == 'prophet':
#                         fold_model = ProphetOptimizedModel(self.season_type)
#                         fold_model.best_params = model.best_params
#                         fold_model.fit(fold_train, target_col)
#                         fold_pred = fold_model.predict(fold_val)
#                     elif name == 'catboost':
#                         fold_model = CatBoostOptimizedModel(self.season_type)
#                         fold_model.best_params = model.best_params
#                         fold_model.fit(fold_train, target_col)
#                         fold_pred = fold_model.predict(fold_val)
#                     # else:  # lstm
#                     #     fold_model = LSTMOptimizedModel(self.season_type)
#                     #     fold_model.best_params = model.best_params
#                     #     # LSTM은 CV splits를 직접 전달하지 않고 fit만 수행
#                     #     fold_model.fit(fold_train, target_col)
#                     #     fold_pred = fold_model.predict(fold_val)
                    
#                     oof_predictions[val_idx] = fold_pred
                    
#                     # GPU 메모리 정리 (LSTM의 경우)
#                     if name == 'lstm' and torch.cuda.is_available():
#                         torch.cuda.empty_cache()

#                 level1_predictions_dict[name] = oof_predictions

#                 # 개별 모델 성능 계산
#                 metrics = evaluate_predictions(train_df[target_col].values, oof_predictions, delta=1.0)
#                 mae = mean_absolute_error(train_df[target_col].values, oof_predictions)
#                 train_time = (datetime.now() - start_time).total_seconds()

#                 self.individual_scores[name] = {
#                     'rmse': metrics['rmse'],
#                     'huber': metrics['huber'], 
#                     'mae': mae, 
#                     'optuna_score': best_score,
#                     'train_time': train_time
#                 }

#                 print(f"   {name} 성능: RMSE={metrics['rmse']:.4f}, Huber={metrics['huber']:.4f}, MAE={mae:.4f}")
#                 print(f"   Optuna 최적 점수: {best_score:.4f}")
#                 print(f"   총 시간: {train_time:.1f}초")

#             except Exception as e:
#                 print(f"❌ {name} 훈련 실패:")
#                 print(f"   에러: {str(e)}")
#                 raise e  # ✅ 에러 발생시키고 중단

#         # 2단계: 메타 모델 최적화 및 훈련
#         print(f"\nRidge 메타 모델 최적화 및 훈련...")
        
#         # 레벨1 피쳐 구성
#         level1_features = np.column_stack(list(level1_predictions_dict.values()))
#         targets = train_df[target_col].values
        
#         print(f"   메타 모델 입력: {level1_features.shape}")
#         print(f"   개별 모델 예측 통계:")
#         for i, (name, pred) in enumerate(level1_predictions_dict.items()):
#             print(f"     {name}: 평균={pred.mean():.2f}, 표준편차={pred.std():.2f}")
        
#         # 메타모델 하이퍼파라미터 최적화
#         meta_score = self.optimize_meta_model(level1_features, targets, cv_splits, optimize_trials['meta'])
        
#         # 최적화된 파라미터로 메타모델 훈련
#         try:
#             self.meta_model = Ridge(
#                 alpha=self.best_meta_params.get('alpha', 1.0),
#                 random_state=SEED
#             )
#             self.meta_model.fit(level1_features, targets)

#             # 스태킹 성능 계산
#             stacking_pred = self.meta_model.predict(level1_features)
#             stacking_pred = np.maximum(stacking_pred, 0)  # 음수 제거
            
#             stacking_metrics = evaluate_predictions(targets, stacking_pred, delta=1.0)
#             stacking_mae = mean_absolute_error(targets, stacking_pred)

#             self.individual_scores['stacking'] = {
#                 'rmse': stacking_metrics['rmse'],
#                 'huber': stacking_metrics['huber'], 
#                 'mae': stacking_mae,
#                 'optuna_score': meta_score
#             }
            
#             print(f"   스태킹 성능: RMSE={stacking_metrics['rmse']:.4f}, Huber={stacking_metrics['huber']:.4f}, MAE={stacking_mae:.4f}")
            
#             # 메타모델 가중치 출력
#             if hasattr(self.meta_model, 'coef_'):
#                 model_names = list(level1_predictions_dict.keys())
#                 print(f"   메타모델 가중치:")
#                 for i, (name, coef) in enumerate(zip(model_names, self.meta_model.coef_)):
#                     print(f"     {name}: {coef:.4f}")
            
#         except Exception as e:
#             print(f"❌ 메타모델 훈련 실패:")
#             print(f"   에러: {str(e)}")
#             raise e  # ✅ 에러 발생시키고 중단
            
#         print(f"✅ {self.group_name} 스태킹 앙상블 훈련 완료!")

#     def predict(self, test_df):
#         """스태킹 앙상블 예측"""
#         if self.meta_model is None:
#             raise RuntimeError(f"{self.group_name} 모델이 훈련되지 않았습니다!")

#         level1_predictions = {}

#         # 1단계: 개별 모델 예측
#         print(f"   {self.group_name} 개별 모델 예측 중...")
#         for name, model in self.models.items():
#             try:
#                 level1_predictions[name] = model.predict(test_df)
#                 pred_stats = level1_predictions[name]
#                 print(f"     {name}: 평균={pred_stats.mean():.2f}, 범위=[{pred_stats.min():.2f}, {pred_stats.max():.2f}]")
#             except Exception as e:
#                 print(f"❌ {name} 예측 실패:")
#                 print(f"   에러: {str(e)}")
#                 raise e  # ✅ 에러 발생시키고 중단

#         # 2단계: 메타 모델 예측
#         try:
#             meta_features = np.column_stack(list(level1_predictions.values()))
#             final_pred = self.meta_model.predict(meta_features)
#             final_pred = np.maximum(final_pred, 0)  # 음수 제거

#             print(f"   {self.group_name} 스태킹 예측 완료: 평균={final_pred.mean():.2f}, 범위=[{final_pred.min():.2f}, {final_pred.max():.2f}]")
            
#             return final_pred, level1_predictions
            
#         except Exception as e:
#             print(f"❌ {self.group_name} 메타모델 예측 실패:")
#             print(f"   에러: {str(e)}")
#             raise e  # ✅ 에러 발생시키고 중단

# print("✅ 고도화된 스태킹 앙상블 클래스 정의 완료")

# # 결과 저장용 딕셔너리
# ensemble_models = {}
# group_results = {}

# print("\n🎯 2개 그룹별 개별 훈련 준비 완료!")

✅ 고도화된 스태킹 앙상블 클래스 정의 완료

🎯 2개 그룹별 개별 훈련 준비 완료!


## 7. 스태킹 앙상블 클래스 (Ridge 메타모델 최적화) v2 - 파라미터 고정

In [20]:
from sklearn.linear_model import Ridge

class AdvancedStackingEnsemble:
    def __init__(self, season_type="heating", group_name=""):
        self.season_type = season_type
        self.group_name = group_name
        self.models = {
            'prophet': ProphetOptimizedModel(season_type),
            'catboost': CatBoostOptimizedModel(season_type)
            # 'lstm': LSTMOptimizedModel(season_type)
        }
        self.meta_model = None
        self.best_meta_params = {}
        self.individual_scores = {}
        
        # 🎯 미리 찾은 최적 파라미터들
        self.predefined_params = self._get_predefined_params()

    
    def _get_predefined_params(self):
        """시즌별 최적 파라미터 반환"""
        if self.season_type == "heating":
            # 난방 시즌 최적 파라미터
            return {
                'prophet': {
                    'changepoint_prior_scale': 0.0026364803038431655,
                    'seasonality_prior_scale': 0.13066739238053282,
                    'holidays_prior_scale': 5.3994844097874335,
                    'seasonality_mode': 'multiplicative'
                },
                'catboost': {
                    'iterations': 1678,
                    'depth': 5,
                    'learning_rate': 0.05748924681991978,
                    'l2_leaf_reg': 12.255876808378806,
                    'border_count': 42
                },
                'ridge': {
                    'alpha': 63.512210106407046
                },
                'expected_rmse': {
                    'ridge': 24.8784
                }
            }
        else:
            # 비난방 시즌 최적 파라미터
            return {
                'prophet': {
                    'changepoint_prior_scale': 0.0010695090612476649,
                    'seasonality_prior_scale': 3.652041851774491,
                    'holidays_prior_scale': 0.45441617690609376,
                    'seasonality_mode': 'additive'
                },
                'catboost': {
                    'iterations': 1818,
                    'depth': 8,
                    'learning_rate': 0.08527855281875678,
                    'l2_leaf_reg': 14.955151393164728,
                    'border_count': 35
                },
                'ridge': {
                    'alpha': 63.512210106407046
                },
                'expected_rmse': {
                    'ridge': 9.8138
                }
            }

    def fit(self, train_df, cv_splits, target_col='heat_demand', use_predefined_params=True):
        """스태킹 앙상블 훈련 (최적 파라미터 사용)"""
        print(f"\n{self.group_name} 스태킹 앙상블 훈련 시작! ({self.season_type} 시즌)")
        print("=" * 60)
        
        if len(train_df) < 100:
            raise RuntimeError(f"데이터가 부족합니다 ({len(train_df)}개). 최소 100개 필요.")

        print(f"훈련 데이터: {len(train_df):,}개")
        print(f"연도 분포: {dict(train_df['year'].value_counts().sort_index())}")
        
        if use_predefined_params:
            print(f"🎯 {self.season_type} 시즌 최적 파라미터 사용")
            print(f"   Prophet: {self.predefined_params['prophet']}")
            print(f"   CatBoost: {self.predefined_params['catboost']}")
            print(f"   Ridge: {self.predefined_params['ridge']}")

        # 1단계: 개별 모델 최적 파라미터 설정 및 훈련
        level1_predictions_dict = {}

        for name, model in self.models.items():
            print(f"\n{name.upper()} 훈련...")
            try:
                start_time = datetime.now()
                
                if use_predefined_params:
                    # 🎯 미리 찾은 최적 파라미터 사용
                    model.best_params = self.predefined_params[name].copy()
                    print(f"   최적 파라미터 적용: {model.best_params}")
                    best_score = self.predefined_params['expected_rmse'].get(name, 0.0)
                else:
                    # 하이퍼파라미터 최적화 (기존 방식)
                    best_score = model.optimize_hyperparameters(
                        train_df, cv_splits, target_col, n_trials=30
                    )
                
                # 최적화된 파라미터로 전체 훈련 데이터에 재훈련
                model.fit(train_df, target_col)
                
                # CV를 통한 레벨1 예측값 생성 (Out-of-Fold 예측)
                oof_predictions = np.zeros(len(train_df))
                
                for fold, (train_idx, val_idx) in enumerate(cv_splits):
                    print(f"   OOF Fold {fold+1} 처리 중...")
                    
                    fold_train = train_df.iloc[train_idx]
                    fold_val = train_df.iloc[val_idx]
                    
                    # 폴드별 모델 훈련 (최적 파라미터 사용)
                    if name == 'prophet':
                        fold_model = ProphetOptimizedModel(self.season_type)
                        fold_model.best_params = model.best_params
                        fold_model.fit(fold_train, target_col)
                        fold_pred = fold_model.predict(fold_val)
                    elif name == 'catboost':
                        fold_model = CatBoostOptimizedModel(self.season_type)
                        fold_model.best_params = model.best_params
                        fold_model.fit(fold_train, target_col)
                        fold_pred = fold_model.predict(fold_val)
                    
                    oof_predictions[val_idx] = fold_pred
                    
                    # GPU 메모리 정리 (필요시)
                    if name == 'lstm' and torch.cuda.is_available():
                        torch.cuda.empty_cache()

                level1_predictions_dict[name] = oof_predictions

                # 개별 모델 성능 계산
                metrics = evaluate_predictions(train_df[target_col].values, oof_predictions, delta=1.0)
                mae = mean_absolute_error(train_df[target_col].values, oof_predictions)
                train_time = (datetime.now() - start_time).total_seconds()

                self.individual_scores[name] = {
                    'rmse': metrics['rmse'],
                    'huber': metrics['huber'], 
                    'mae': mae, 
                    'optuna_score': best_score,
                    'train_time': train_time
                }

                print(f"   {name} 성능: RMSE={metrics['rmse']:.4f}, Huber={metrics['huber']:.4f}, MAE={mae:.4f}")
                if use_predefined_params:
                    print(f"   예상 점수 대비: {best_score:.4f}")
                else:
                    print(f"   Optuna 최적 점수: {best_score:.4f}")
                print(f"   총 시간: {train_time:.1f}초")

            except Exception as e:
                print(f"❌ {name} 훈련 실패:")
                print(f"   에러: {str(e)}")
                raise e

        # 2단계: 메타 모델 훈련
        print(f"\nRidge 메타 모델 훈련...")
        
        # 레벨1 피쳐 구성
        level1_features = np.column_stack(list(level1_predictions_dict.values()))
        targets = train_df[target_col].values
        
        print(f"   메타 모델 입력: {level1_features.shape}")
        print(f"   개별 모델 예측 통계:")
        for i, (name, pred) in enumerate(level1_predictions_dict.items()):
            print(f"     {name}: 평균={pred.mean():.2f}, 표준편차={pred.std():.2f}")
        
        try:
            if use_predefined_params:
                # 🎯 미리 찾은 최적 파라미터 사용
                self.best_meta_params = self.predefined_params['ridge'].copy()
                meta_score = self.predefined_params['expected_rmse']['ridge']
                print(f"   최적 Ridge 파라미터 적용: {self.best_meta_params}")
                print(f"   예상 RMSE: {meta_score:.4f}")
            else:
                # 메타모델 하이퍼파라미터 최적화 (기존 방식)
                meta_score = self.optimize_meta_model(level1_features, targets, cv_splits, 20)
            
            # 최적화된 파라미터로 메타모델 훈련
            self.meta_model = Ridge(
                alpha=self.best_meta_params.get('alpha', 1.0),
                random_state=SEED
            )
            self.meta_model.fit(level1_features, targets)

            # 스태킹 성능 계산
            stacking_pred = self.meta_model.predict(level1_features)
            stacking_pred = np.maximum(stacking_pred, 0)  # 음수 제거
            
            stacking_metrics = evaluate_predictions(targets, stacking_pred, delta=1.0)
            stacking_mae = mean_absolute_error(targets, stacking_pred)

            self.individual_scores['stacking'] = {
                'rmse': stacking_metrics['rmse'],
                'huber': stacking_metrics['huber'], 
                'mae': stacking_mae,
                'optuna_score': meta_score
            }
            
            print(f"   스태킹 성능: RMSE={stacking_metrics['rmse']:.4f}, Huber={stacking_metrics['huber']:.4f}, MAE={stacking_mae:.4f}")
            
            # 메타모델 가중치 출력
            if hasattr(self.meta_model, 'coef_'):
                model_names = list(level1_predictions_dict.keys())
                print(f"   메타모델 가중치:")
                for i, (name, coef) in enumerate(zip(model_names, self.meta_model.coef_)):
                    print(f"     {name}: {coef:.4f}")
            
        except Exception as e:
            print(f"❌ 메타모델 훈련 실패:")
            print(f"   에러: {str(e)}")
            raise e
            
        print(f"✅ {self.group_name} 스태킹 앙상블 훈련 완료!")

    def optimize_meta_model(self, level1_features, targets, cv_splits, n_trials=20):
        """Ridge 메타모델 최적화 (기존 코드 유지 - 필요시 사용)"""
        print(f"Ridge 메타모델 최적화 중... (trials: {n_trials})")
        
        def objective(trial):
            alpha = trial.suggest_float('alpha', 0.01, 100.0, log=True)
            
            cv_scores = []
            
            for fold, (train_idx, val_idx) in enumerate(cv_splits):
                try:
                    train_meta_mask = np.isin(range(len(level1_features)), train_idx)
                    val_meta_mask = np.isin(range(len(level1_features)), val_idx)
                    
                    X_train_meta = level1_features[train_meta_mask]
                    X_val_meta = level1_features[val_meta_mask]
                    y_train_meta = targets[train_meta_mask]
                    y_val_meta = targets[val_meta_mask]
                    
                    if len(X_train_meta) < 5 or len(X_val_meta) < 2:
                        print(f"     메타 Fold {fold+1}: 데이터 부족")
                        continue
                    
                    model = Ridge(alpha=alpha, random_state=SEED)
                    model.fit(X_train_meta, y_train_meta)
                    
                    pred = model.predict(X_val_meta)
                    pred = np.maximum(pred, 0)
                    
                    rmse = np.sqrt(mean_squared_error(y_val_meta, pred))
                    cv_scores.append(rmse)
                    print(f"     메타 Fold {fold+1}: RMSE = {rmse:.4f}")
                    
                except Exception as e:
                    print(f"❌ 메타 Fold {fold+1} 최적화 실패:")
                    print(f"   에러: {str(e)}")
                    raise e
                    
            if len(cv_scores) == 0:
                print(f"   ⚠️ 모든 메타 Fold에서 최적화 실패")
                return 999.0
                
            return np.mean(cv_scores)
        
        study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        self.best_meta_params = study.best_params
        print(f"   Ridge 최적 RMSE: {study.best_value:.4f}")
        print(f"   최적 파라미터: {self.best_meta_params}")
        
        return study.best_value

    def predict(self, test_df):
        """스태킹 앙상블 예측"""
        if self.meta_model is None:
            raise RuntimeError(f"{self.group_name} 모델이 훈련되지 않았습니다!")

        level1_predictions = {}

        # 1단계: 개별 모델 예측
        print(f"   {self.group_name} 개별 모델 예측 중...")
        for name, model in self.models.items():
            try:
                level1_predictions[name] = model.predict(test_df)
                pred_stats = level1_predictions[name]
                print(f"     {name}: 평균={pred_stats.mean():.2f}, 범위=[{pred_stats.min():.2f}, {pred_stats.max():.2f}]")
            except Exception as e:
                print(f"❌ {name} 예측 실패:")
                print(f"   에러: {str(e)}")
                raise e

        # 2단계: 메타 모델 예측
        try:
            meta_features = np.column_stack(list(level1_predictions.values()))
            final_pred = self.meta_model.predict(meta_features)
            final_pred = np.maximum(final_pred, 0)  # 음수 제거

            print(f"   {self.group_name} 스태킹 예측 완료: 평균={final_pred.mean():.2f}, 범위=[{final_pred.min():.2f}, {final_pred.max():.2f}]")
            
            return final_pred, level1_predictions
            
        except Exception as e:
            print(f"❌ {self.group_name} 메타모델 예측 실패:")
            print(f"   에러: {str(e)}")
            raise e

print("✅ 최적 파라미터 적용된 스태킹 앙상블 클래스 정의 완료")

# 결과 저장용 딕셔너리
ensemble_models = {}
group_results = {}

print("\n🎯 최적 파라미터로 2개 그룹별 개별 훈련 준비 완료!")

# 사용 예시
"""
# 난방 시즌 모델
heating_ensemble = AdvancedStackingEnsemble(season_type="heating", group_name="난방시즌")
heating_ensemble.fit(heating_train_df, heating_cv_splits, use_predefined_params=True)

# 비난방 시즌 모델  
non_heating_ensemble = AdvancedStackingEnsemble(season_type="non_heating", group_name="비난방시즌")
non_heating_ensemble.fit(non_heating_train_df, non_heating_cv_splits, use_predefined_params=True)
"""

✅ 최적 파라미터 적용된 스태킹 앙상블 클래스 정의 완료

🎯 최적 파라미터로 2개 그룹별 개별 훈련 준비 완료!


'\n# 난방 시즌 모델\nheating_ensemble = AdvancedStackingEnsemble(season_type="heating", group_name="난방시즌")\nheating_ensemble.fit(heating_train_df, heating_cv_splits, use_predefined_params=True)\n\n# 비난방 시즌 모델  \nnon_heating_ensemble = AdvancedStackingEnsemble(season_type="non_heating", group_name="비난방시즌")\nnon_heating_ensemble.fit(non_heating_train_df, non_heating_cv_splits, use_predefined_params=True)\n'

## 8. 그룹별 개별 훈련

### (현재) Prophet

"전체 기준으로 하이퍼파라미터를 찾은 후, 지사별로 학습" ✅

하이퍼파라미터: 모든 지사 통합 성능으로 최적화
모델 훈련: 찾은 파라미터로 지사별 개별 모델 생성

2. 실제로는

하나의 파라미터 조합을 모든 지사에 적용해서 테스트
지사별 성능을 종합해서 그 파라미터 조합의 점수 계산
30번 반복해서 가장 좋은 파라미터 조합 찾기

3. 최종 결과

전체적으로 가장 좋은 하나의 파라미터 세트 선택
이 파라미터를 모든 지사에 동일하게 적용해서 개별 모델 훈련

In [19]:
# # 모든 그룹 훈련 함수
# def train_all_groups():
#     """2개 그룹 훈련 (난방/비난방) - 연도 기반 CV 적용"""
#     group_configs = {
#         "heating": {
#             "season": "heating", 
#             # "trials": {"prophet": 40, "catboost": 60, "meta": 25}
#             "trials": {"prophet": 1, "catboost": 1, "meta": 1}
#         },
#         "non_heating": {
#             "season": "non_heating", 
#             # "trials": {"prophet": 30, "catboost": 50, "meta": 20}
#             "trials": {"prophet": 1, "catboost": 1,  "meta": 1}
#         }
#     }
    
#     print("🚀 시즌별 2개 그룹 훈련 시작 (연도 기반 3-Fold CV)")
#     total_start_time = datetime.now()
    
#     for group_name, config in group_configs.items():
#         print(f"\n{'='*60}")
#         print(f"🔥 {group_name.upper()} 그룹 훈련")
        
#         # 그룹 데이터 검증
#         if group_name not in train_groups:
#             raise KeyError(f"'{group_name}' 그룹이 train_groups에 없습니다.")
            
#         group_data = train_groups[group_name]
        
#         if len(group_data) == 0:
#             raise ValueError(f"{group_name} 그룹에 데이터가 없습니다.")
        
#         if 'heat_demand' not in group_data.columns:
#             raise ValueError(f"{group_name} 그룹에 'heat_demand' 컬럼이 없습니다.")
        
#         print(f"📊 데이터 크기: {len(group_data):,}개")
#         print(f"🏢 지사 수: {group_data['branch_id'].nunique()}개")
#         print(f"📅 연도 분포: {dict(group_data['year'].value_counts().sort_index())}")
#         print(f"🎯 타겟 통계: 평균={group_data['heat_demand'].mean():.2f}, 표준편차={group_data['heat_demand'].std():.2f}")
        
#         # 최소 데이터 요구량 확인
#         if len(group_data) < 1000:
#             raise ValueError(f"{group_name} 데이터가 부족합니다 ({len(group_data):,}개). 최소 1,000개 필요.")
        
#         # 그룹별 CV 분할 생성
#         print(f"🔄 {group_name} CV 분할 생성 중...")
#         group_cv_splits = create_year_based_cv_splits(group_data, group_name)
        
#         # 앙상블 모델 생성
#         print(f"🏗️ {group_name} 앙상블 모델 생성 중...")
#         ensemble_models[group_name] = AdvancedStackingEnsemble(
#             season_type=config["season"], 
#             group_name=group_name
#         )
        
#         # 훈련 실행
#         print(f"🚀 {group_name} 훈련 시작...")
#         start_time = datetime.now()
        
#         ensemble_models[group_name].fit(
#             group_data, 
#             group_cv_splits,
#             target_col='heat_demand',
#             optimize_trials=config["trials"]
#         )
        
#         total_time = (datetime.now() - start_time).total_seconds()
        
#         # 결과 저장
#         group_results[group_name] = {
#             'scores': ensemble_models[group_name].individual_scores.copy(),
#             'total_time': total_time,
#             'data_size': len(group_data),
#             'branch_count': group_data['branch_id'].nunique(),
#             'year_distribution': dict(group_data['year'].value_counts().sort_index())
#         }
        
#         # 성능 결과 출력 (Huber Loss 포함)
#         print(f"\n📈 {group_name} 최종 결과:")
#         print(f"{'모델':12s} {'RMSE':>8s} {'Huber':>8s} {'MAE':>8s} {'시간(초)':>8s}")
#         print("-" * 50)
        
#         for model, scores in ensemble_models[group_name].individual_scores.items():
#             rmse = scores.get('rmse', 999)
#             huber = scores.get('huber', 999)
#             mae = scores.get('mae', 999)
#             model_time = scores.get('train_time', 0)
            
#             print(f"{model:12s} {rmse:8.4f} {huber:8.4f} {mae:8.4f} {model_time:8.1f}")
        
#         print(f"   ⏱️ 총 훈련 시간: {total_time:.1f}초 ({total_time/60:.1f}분)")
        
#         # 최고 성능 모델 확인
#         best_model = min(
#             [(name, score.get('huber', 999)) for name, score in ensemble_models[group_name].individual_scores.items()],
#             key=lambda x: x[1]
#         )
#         print(f"   🏆 최고 성능: {best_model[0]} (Huber Loss: {best_model[1]:.4f})")
#         print(f"✅ {group_name} 그룹 훈련 완료!")
    
#     # 전체 결과 요약
#     total_training_time = (datetime.now() - total_start_time).total_seconds()
    
#     print(f"\n🎉 전체 훈련 완료!")
#     print(f"=" * 60)
#     print(f"⏱️ 총 훈련 시간: {total_training_time/60:.1f}분 ({total_training_time/3600:.1f}시간)")
    
#     # 전체 평균 성능
#     print(f"\n📊 전체 평균 성능:")
#     avg_scores = {'rmse': [], 'huber': [], 'mae': []}
    
#     for group_name, result in group_results.items():
#         if result is not None and 'scores' in result:
#             stacking_score = result['scores'].get('stacking', {})
#             for metric in avg_scores.keys():
#                 if metric in stacking_score:
#                     avg_scores[metric].append(stacking_score[metric])
    
#     for metric, scores in avg_scores.items():
#         if scores:
#             print(f"   평균 {metric.upper()}: {np.mean(scores):.4f} (±{np.std(scores):.4f})")
    
#     # GPU 메모리 정리
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#         print(f"🧹 GPU 메모리 정리 완료")

# # 모든 그룹 훈련 실행
# train_all_groups()

🚀 시즌별 2개 그룹 훈련 시작 (연도 기반 3-Fold CV)

🔥 HEATING 그룹 훈련
📊 데이터 크기: 289,997개
🏢 지사 수: 19개
📅 연도 분포: {2021: 96653, 2022: 96672, 2023: 96672}
🎯 타겟 통계: 평균=135.94, 표준편차=135.29
🔄 heating CV 분할 생성 중...
heating 그룹 - 연도 기반 3-Fold CV 분할 생성...
   연도별 데이터 분포:
     2021년: 96,653개
     2022년: 96,672개
     2023년: 96,672개
   Fold 2021: 훈련 193,344개, 검증 96,653개
   Fold 2022: 훈련 193,325개, 검증 96,672개
   Fold 2023: 훈련 193,325개, 검증 96,672개
🏗️ heating 앙상블 모델 생성 중...
🚀 heating 훈련 시작...

heating 스태킹 앙상블 훈련 시작!
훈련 데이터: 289,997개
연도 분포: {2021: 96653, 2022: 96672, 2023: 96672}

PROPHET 최적화 및 훈련...
Prophet Huber Loss 하이퍼파라미터 최적화 중... (trials: 1)


  0%|          | 0/1 [00:00<?, ?it/s]

[W 2025-06-24 11:03:40,374] Trial 0 failed with parameters: {'changepoint_prior_scale': 0.010253509690168494, 'seasonality_prior_scale': 7.969454818643936, 'holidays_prior_scale': 2.9106359131330697, 'seasonality_mode': 'additive'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/jisupark_1/workspace/star_track_python/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/pt/8357krnj4tv48kdjx9mrhd3r0000gp/T/ipykernel_36917/2016375091.py", line 68, in objective
    model.fit(prophet_df)
  File "/Users/jisupark_1/workspace/star_track_python/.venv/lib/python3.10/site-packages/prophet/forecaster.py", line 1235, in fit
    self.params = self.stan_backend.fit(stan_init, dat, **kwargs)
  File "/Users/jisupark_1/workspace/star_track_python/.venv/lib/python3.10/site-packages/prophet/models.py", line 126, in fit
    self.stan_fit = self.model.optimize(**arg

KeyboardInterrupt: 

## 8-2. Train 

In [21]:
# 모든 그룹 훈련 함수 (수정된 버전)
def train_all_groups():
    """2개 그룹 훈련 (난방/비난방) - 연도 기반 CV 적용"""
    group_configs = {
        "heating": {
            "season": "heating", 
            "use_predefined": True  # 🎯 최적 파라미터 사용
        },
        "non_heating": {
            "season": "non_heating", 
            "use_predefined": True  # 🎯 최적 파라미터 사용
        }
    }
    
    print("🚀 시즌별 2개 그룹 훈련 시작 (연도 기반 3-Fold CV)")
    print("🎯 미리 찾은 최적 파라미터 사용으로 빠른 훈련!")
    total_start_time = datetime.now()
    
    for group_name, config in group_configs.items():
        print(f"\n{'='*60}")
        print(f"🔥 {group_name.upper()} 그룹 훈련")
        
        # 그룹 데이터 검증
        if group_name not in train_groups:
            raise KeyError(f"'{group_name}' 그룹이 train_groups에 없습니다.")
            
        group_data = train_groups[group_name]
        
        if len(group_data) == 0:
            raise ValueError(f"{group_name} 그룹에 데이터가 없습니다.")
        
        if 'heat_demand' not in group_data.columns:
            raise ValueError(f"{group_name} 그룹에 'heat_demand' 컬럼이 없습니다.")
        
        print(f"📊 데이터 크기: {len(group_data):,}개")
        print(f"🏢 지사 수: {group_data['branch_id'].nunique()}개")
        print(f"📅 연도 분포: {dict(group_data['year'].value_counts().sort_index())}")
        print(f"🎯 타겟 통계: 평균={group_data['heat_demand'].mean():.2f}, 표준편차={group_data['heat_demand'].std():.2f}")
        
        # 최소 데이터 요구량 확인
        if len(group_data) < 1000:
            raise ValueError(f"{group_name} 데이터가 부족합니다 ({len(group_data):,}개). 최소 1,000개 필요.")
        
        # 그룹별 CV 분할 생성
        print(f"🔄 {group_name} CV 분할 생성 중...")
        group_cv_splits = create_year_based_cv_splits(group_data, group_name)
        
        # 앙상블 모델 생성
        print(f"🏗️ {group_name} 앙상블 모델 생성 중...")
        ensemble_models[group_name] = AdvancedStackingEnsemble(
            season_type=config["season"], 
            group_name=group_name
        )
        
        # 훈련 실행 (🎯 수정된 부분)
        print(f"🚀 {group_name} 훈련 시작...")
        start_time = datetime.now()
        
        ensemble_models[group_name].fit(
            group_data, 
            group_cv_splits,
            target_col='heat_demand',
            use_predefined_params=config["use_predefined"]  # 🎯 변경됨
        )
        
        total_time = (datetime.now() - start_time).total_seconds()
        
        # 결과 저장
        group_results[group_name] = {
            'scores': ensemble_models[group_name].individual_scores.copy(),
            'total_time': total_time,
            'data_size': len(group_data),
            'branch_count': group_data['branch_id'].nunique(),
            'year_distribution': dict(group_data['year'].value_counts().sort_index())
        }
        
        # 성능 결과 출력 (Huber Loss 포함)
        print(f"\n📈 {group_name} 최종 결과:")
        print(f"{'모델':12s} {'RMSE':>8s} {'Huber':>8s} {'MAE':>8s} {'시간(초)':>8s}")
        print("-" * 50)
        
        for model, scores in ensemble_models[group_name].individual_scores.items():
            rmse = scores.get('rmse', 999)
            huber = scores.get('huber', 999)
            mae = scores.get('mae', 999)
            model_time = scores.get('train_time', 0)
            
            print(f"{model:12s} {rmse:8.4f} {huber:8.4f} {mae:8.4f} {model_time:8.1f}")
        
        print(f"   ⏱️ 총 훈련 시간: {total_time:.1f}초 ({total_time/60:.1f}분)")
        
        # 최고 성능 모델 확인
        best_model = min(
            [(name, score.get('huber', 999)) for name, score in ensemble_models[group_name].individual_scores.items()],
            key=lambda x: x[1]
        )
        print(f"   🏆 최고 성능: {best_model[0]} (Huber Loss: {best_model[1]:.4f})")
        print(f"✅ {group_name} 그룹 훈련 완료!")
    
    # 전체 결과 요약
    total_training_time = (datetime.now() - total_start_time).total_seconds()
    
    print(f"\n🎉 전체 훈련 완료!")
    print(f"=" * 60)
    print(f"⏱️ 총 훈련 시간: {total_training_time/60:.1f}분 ({total_training_time/3600:.1f}시간)")
    
    # 전체 평균 성능
    print(f"\n📊 전체 평균 성능:")
    avg_scores = {'rmse': [], 'huber': [], 'mae': []}
    
    for group_name, result in group_results.items():
        if result is not None and 'scores' in result:
            stacking_score = result['scores'].get('stacking', {})
            for metric in avg_scores.keys():
                if metric in stacking_score:
                    avg_scores[metric].append(stacking_score[metric])
    
    for metric, scores in avg_scores.items():
        if scores:
            print(f"   평균 {metric.upper()}: {np.mean(scores):.4f} (±{np.std(scores):.4f})")
    
    # 🎯 최적 파라미터 사용 결과 요약
    print(f"\n🎯 사용된 최적 파라미터:")
    for group_name in group_configs.keys():
        if group_name in ensemble_models:
            model = ensemble_models[group_name]
            print(f"\n{group_name.upper()} 시즌:")
            print(f"   Prophet: {model.predefined_params['prophet']}")
            print(f"   CatBoost: {model.predefined_params['catboost']}")
            print(f"   Ridge: {model.predefined_params['ridge']}")
    
    # GPU 메모리 정리
    if 'torch' in globals() and torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"🧹 GPU 메모리 정리 완료")

# 🚀 모든 그룹 훈련 실행
print("🎯 최적 파라미터를 사용한 빠른 훈련 시작!")
train_all_groups()

🎯 최적 파라미터를 사용한 빠른 훈련 시작!
🚀 시즌별 2개 그룹 훈련 시작 (연도 기반 3-Fold CV)
🎯 미리 찾은 최적 파라미터 사용으로 빠른 훈련!

🔥 HEATING 그룹 훈련
📊 데이터 크기: 289,997개
🏢 지사 수: 19개
📅 연도 분포: {2021: 96653, 2022: 96672, 2023: 96672}
🎯 타겟 통계: 평균=135.94, 표준편차=135.29
🔄 heating CV 분할 생성 중...
heating 그룹 - 연도 기반 3-Fold CV 분할 생성...
   연도별 데이터 분포:
     2021년: 96,653개
     2022년: 96,672개
     2023년: 96,672개
   Fold 2021: 훈련 193,344개, 검증 96,653개
   Fold 2022: 훈련 193,325개, 검증 96,672개
   Fold 2023: 훈련 193,325개, 검증 96,672개
🏗️ heating 앙상블 모델 생성 중...
🚀 heating 훈련 시작...

heating 스태킹 앙상블 훈련 시작! (heating 시즌)
훈련 데이터: 289,997개
연도 분포: {2021: 96653, 2022: 96672, 2023: 96672}
🎯 heating 시즌 최적 파라미터 사용
   Prophet: {'changepoint_prior_scale': 0.0026364803038431655, 'seasonality_prior_scale': 0.13066739238053282, 'holidays_prior_scale': 5.3994844097874335, 'seasonality_mode': 'multiplicative'}
   CatBoost: {'iterations': 1678, 'depth': 5, 'learning_rate': 0.05748924681991978, 'l2_leaf_reg': 12.255876808378806, 'border_count': 42}
   Ridge: {'alpha': 63.512210

Prophet 지사별 훈련:   0%|          | 0/19 [00:00<?, ?it/s]

   19/19개 지사 훈련 완료
   OOF Fold 1 처리 중...
Prophet 모델 훈련 중...
   사용할 regressors: ['ta', 'hm', 'ws', 'HDD18', 'apparent_temp', 'apparent_temp', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'ta_lag_3h', 'ta_lag_6h', 'heating_month_order']


Prophet 지사별 훈련:   0%|          | 0/19 [00:00<?, ?it/s]

   19/19개 지사 훈련 완료
   OOF Fold 2 처리 중...
Prophet 모델 훈련 중...
   사용할 regressors: ['ta', 'hm', 'ws', 'HDD18', 'apparent_temp', 'apparent_temp', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'ta_lag_3h', 'ta_lag_6h', 'heating_month_order']


...

⚠️ 지사 A: 누락된 regressors: ['apparent_temp']
⚠️ 지사 B: 누락된 regressors: ['apparent_temp']
⚠️ 지사 C: 누락된 regressors: ['apparent_temp']
⚠️ 지사 D: 누락된 regressors: ['apparent_temp']
⚠️ 지사 E: 누락된 regressors: ['apparent_temp']
⚠️ 지사 F: 누락된 regressors: ['apparent_temp']
⚠️ 지사 G: 누락된 regressors: ['apparent_temp']
⚠️ 지사 H: 누락된 regressors: ['apparent_temp']
⚠️ 지사 I: 누락된 regressors: ['apparent_temp']
⚠️ 지사 J: 누락된 regressors: ['apparent_temp']
⚠️ 지사 K: 누락된 regressors: ['apparent_temp']
⚠️ 지사 L: 누락된 regressors: ['apparent_temp']
⚠️ 지사 M: 누락된 regressors: ['apparent_temp']
⚠️ 지사 N: 누락된 regressors: ['apparent_temp']
⚠️ 지사 O: 누락된 regressors: ['apparent_temp']
⚠️ 지사 P: 누락된 regressors: ['apparent_temp']
⚠️ 지사 Q: 누락된 regressors: ['apparent_temp']
⚠️ 지사 R: 누락된 regressors: ['apparent_temp']
⚠️ 지사 S: 누락된 regressors: ['apparent_temp']
   19/19개 지사 훈련 완료
   OOF Fold 2 처리 중...
Prophet 모델 훈련 중...
   사용할 regressors: ['ta', 'hm', 'ws', 'HDD18', 'apparent_temp', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofwee

Prophet 지사별 훈련:   0%|          | 0/19 [00:00<?, ?it/s]

⚠️ 지사 A: 누락된 regressors: ['apparent_temp']
⚠️ 지사 B: 누락된 regressors: ['apparent_temp']
⚠️ 지사 C: 누락된 regressors: ['apparent_temp']
⚠️ 지사 D: 누락된 regressors: ['apparent_temp']
⚠️ 지사 E: 누락된 regressors: ['apparent_temp']
⚠️ 지사 F: 누락된 regressors: ['apparent_temp']
⚠️ 지사 G: 누락된 regressors: ['apparent_temp']
⚠️ 지사 H: 누락된 regressors: ['apparent_temp']
⚠️ 지사 I: 누락된 regressors: ['apparent_temp']
⚠️ 지사 J: 누락된 regressors: ['apparent_temp']
⚠️ 지사 K: 누락된 regressors: ['apparent_temp']
⚠️ 지사 L: 누락된 regressors: ['apparent_temp']
⚠️ 지사 M: 누락된 regressors: ['apparent_temp']
⚠️ 지사 N: 누락된 regressors: ['apparent_temp']
⚠️ 지사 O: 누락된 regressors: ['apparent_temp']
⚠️ 지사 P: 누락된 regressors: ['apparent_temp']
⚠️ 지사 Q: 누락된 regressors: ['apparent_temp']
⚠️ 지사 R: 누락된 regressors: ['apparent_temp']
⚠️ 지사 S: 누락된 regressors: ['apparent_temp']
   19/19개 지사 훈련 완료
   OOF Fold 3 처리 중...
Prophet 모델 훈련 중...
   사용할 regressors: ['ta', 'hm', 'ws', 'HDD18', 'apparent_temp', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofwee

Prophet 지사별 훈련:   0%|          | 0/19 [00:00<?, ?it/s]

⚠️ 지사 A: 누락된 regressors: ['apparent_temp']
⚠️ 지사 B: 누락된 regressors: ['apparent_temp']
⚠️ 지사 C: 누락된 regressors: ['apparent_temp']
⚠️ 지사 D: 누락된 regressors: ['apparent_temp']
⚠️ 지사 E: 누락된 regressors: ['apparent_temp']
⚠️ 지사 F: 누락된 regressors: ['apparent_temp']
⚠️ 지사 G: 누락된 regressors: ['apparent_temp']
⚠️ 지사 H: 누락된 regressors: ['apparent_temp']
⚠️ 지사 I: 누락된 regressors: ['apparent_temp']
⚠️ 지사 J: 누락된 regressors: ['apparent_temp']
⚠️ 지사 K: 누락된 regressors: ['apparent_temp']
⚠️ 지사 L: 누락된 regressors: ['apparent_temp']
⚠️ 지사 M: 누락된 regressors: ['apparent_temp']
⚠️ 지사 N: 누락된 regressors: ['apparent_temp']
⚠️ 지사 O: 누락된 regressors: ['apparent_temp']
⚠️ 지사 P: 누락된 regressors: ['apparent_temp']
⚠️ 지사 Q: 누락된 regressors: ['apparent_temp']
⚠️ 지사 R: 누락된 regressors: ['apparent_temp']
⚠️ 지사 S: 누락된 regressors: ['apparent_temp']
   19/19개 지사 훈련 완료
   prophet 성능: RMSE=11.9944, Huber=7.7240, MAE=8.2030
   예상 점수 대비: 0.0000
   총 시간: 100.2초

CATBOOST 훈련...
   최적 파라미터 적용: {'iterations': 1818, 'depth': 8, 'learning_

### 모델 저장 및 로드

In [32]:
def save_trained_models(ensemble_models, group_results, model_save_path="./saved_models/"):
    """이미 훈련된 모델들을 저장하는 함수"""
    import os
    import pickle
    import joblib
    from datetime import datetime
    
    print(f"💾 훈련된 모델 저장 시작...")
    print(f"📁 저장 경로: {model_save_path}")
    
    # 저장 디렉토리 생성
    os.makedirs(model_save_path, exist_ok=True)
    
    # 저장할 모델 확인
    print(f"🔍 저장할 모델 확인:")
    print(f"   ensemble_models: {list(ensemble_models.keys()) if ensemble_models else '없음'}")
    print(f"   group_results: {list(group_results.keys()) if group_results else '없음'}")
    
    saved_files = []
    
    # 각 그룹별 모델 저장
    for group_name in ensemble_models.keys():
        print(f"\n💾 {group_name.upper()} 모델 저장 중...")
        
        try:
            # 1. 전체 앙상블 모델 저장 (pickle)
            ensemble_file = os.path.join(model_save_path, f"ensemble_{group_name}.pkl")
            with open(ensemble_file, 'wb') as f:
                pickle.dump(ensemble_models[group_name], f)
            saved_files.append(ensemble_file)
            print(f"   ✅ 앙상블 모델: ensemble_{group_name}.pkl")
            
            # 2. 개별 모델별 저장
            ensemble_model = ensemble_models[group_name]
            
            for model_name, model in ensemble_model.models.items():
                try:
                    if model_name == 'prophet':
                        # Prophet 모델은 joblib로 저장
                        prophet_file = os.path.join(model_save_path, f"prophet_{group_name}.joblib")
                        joblib.dump(model, prophet_file)
                        saved_files.append(prophet_file)
                        print(f"   ✅ Prophet: prophet_{group_name}.joblib")
                        
                    elif model_name == 'catboost':
                        # CatBoost 모델 저장 시도
                        try:
                            if hasattr(model, 'model') and model.model is not None:
                                # CatBoost 전용 형식으로 저장
                                catboost_file = os.path.join(model_save_path, f"catboost_{group_name}.cbm")
                                model.model.save_model(catboost_file)
                                saved_files.append(catboost_file)
                                print(f"   ✅ CatBoost 모델: catboost_{group_name}.cbm")
                            else:
                                # 전체 객체를 joblib로 저장
                                catboost_file = os.path.join(model_save_path, f"catboost_{group_name}.joblib")
                                joblib.dump(model, catboost_file)
                                saved_files.append(catboost_file)
                                print(f"   ✅ CatBoost 객체: catboost_{group_name}.joblib")
                        except Exception as e:
                            # 실패시 joblib로 백업 저장
                            catboost_file = os.path.join(model_save_path, f"catboost_{group_name}.joblib")
                            joblib.dump(model, catboost_file)
                            saved_files.append(catboost_file)
                            print(f"   ✅ CatBoost 백업: catboost_{group_name}.joblib")
                            
                except Exception as e:
                    print(f"   ⚠️ {model_name} 개별 저장 실패: {str(e)[:50]}...")
            
            # 3. 메타 모델 (Ridge) 저장
            if hasattr(ensemble_model, 'meta_model') and ensemble_model.meta_model is not None:
                ridge_file = os.path.join(model_save_path, f"ridge_{group_name}.joblib")
                joblib.dump(ensemble_model.meta_model, ridge_file)
                saved_files.append(ridge_file)
                print(f"   ✅ Ridge 메타모델: ridge_{group_name}.joblib")
            
            # 4. 최적 파라미터 저장
            if hasattr(ensemble_model, 'predefined_params'):
                params_file = os.path.join(model_save_path, f"params_{group_name}.pkl")
                with open(params_file, 'wb') as f:
                    pickle.dump(ensemble_model.predefined_params, f)
                saved_files.append(params_file)
                print(f"   ✅ 최적 파라미터: params_{group_name}.pkl")
                
        except Exception as e:
            print(f"   ❌ {group_name} 모델 저장 실패: {str(e)}")
    
    # 5. 성능 결과 저장
    if group_results:
        print(f"\n💾 성능 결과 저장 중...")
        for group_name, result in group_results.items():
            try:
                results_file = os.path.join(model_save_path, f"results_{group_name}.pkl")
                with open(results_file, 'wb') as f:
                    pickle.dump(result, f)
                saved_files.append(results_file)
                print(f"   ✅ {group_name} 결과: results_{group_name}.pkl")
            except Exception as e:
                print(f"   ⚠️ {group_name} 결과 저장 실패: {str(e)}")
    
    # 6. 전체 모델 패키지 저장
    try:
        print(f"\n💾 전체 모델 패키지 저장 중...")
        
        full_package = {
            'ensemble_models': ensemble_models,
            'group_results': group_results,
            'training_info': {
                'save_date': datetime.now().isoformat(),
                'model_groups': list(ensemble_models.keys()),
                'total_models': len(ensemble_models)
            }
        }
        
        package_file = os.path.join(model_save_path, "full_model_package.pkl")
        with open(package_file, 'wb') as f:
            pickle.dump(full_package, f)
        saved_files.append(package_file)
        
        print(f"   ✅ 전체 패키지: full_model_package.pkl")
        
    except Exception as e:
        print(f"   ❌ 전체 패키지 저장 실패: {str(e)}")
    
    # 저장 완료 요약
    print(f"\n🎉 모델 저장 완료!")
    print(f"=" * 50)
    print(f"📁 저장 경로: {model_save_path}")
    print(f"📊 저장된 파일 수: {len(saved_files)}개")
    
    # 파일 목록과 크기 출력
    print(f"\n📋 저장된 파일 목록:")
    total_size = 0
    for file_path in saved_files:
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path) / (1024*1024)  # MB
            total_size += file_size
            file_name = os.path.basename(file_path)
            print(f"   {file_name:<30} {file_size:>6.1f}MB")
    
    print(f"\n💾 총 저장 용량: {total_size:.1f}MB")
    
    return saved_files

# 🚀 이미 훈련된 모델 저장 실행
print("💾 현재 훈련된 모델들을 저장합니다...")

# 모델이 존재하는지 확인
if 'ensemble_models' in globals() and ensemble_models:
    if 'group_results' in globals() and group_results:
        saved_files = save_trained_models(ensemble_models, group_results, "./saved_models/")
        print(f"\n✅ 저장 완료! {len(saved_files)}개 파일이 저장되었습니다.")
    else:
        print("⚠️ group_results가 없어서 모델만 저장합니다.")
        saved_files = save_trained_models(ensemble_models, {}, "./saved_models/")
else:
    print("❌ 저장할 훈련된 모델이 없습니다!")
    print("   먼저 train_all_groups()를 실행하여 모델을 훈련하세요.")

# 📥 로드 함수 (참고용)
def load_saved_models(model_save_path="./saved_models/"):
    """저장된 모델들을 로드하는 함수"""
    import pickle
    import os
    
    print(f"📥 저장된 모델 로드 중... ({model_save_path})")
    
    # 전체 패키지 로드 시도
    package_file = os.path.join(model_save_path, "full_model_package.pkl")
    if os.path.exists(package_file):
        try:
            with open(package_file, 'rb') as f:
                package = pickle.load(f)
            
            print("✅ 전체 모델 패키지 로드 성공!")
            print(f"   저장 일시: {package['training_info']['save_date']}")
            print(f"   모델 그룹: {package['training_info']['model_groups']}")
            
            return package['ensemble_models'], package['group_results']
            
        except Exception as e:
            print(f"❌ 전체 패키지 로드 실패: {e}")
    
    print("❌ 저장된 모델을 찾을 수 없습니다.")
    return {}, {}

print(f"\n💡 나중에 모델을 로드하려면:")
print(f"   ensemble_models, group_results = load_saved_models('./saved_models/')")

💾 현재 훈련된 모델들을 저장합니다...
💾 훈련된 모델 저장 시작...
📁 저장 경로: ./saved_models/
🔍 저장할 모델 확인:
   ensemble_models: ['heating', 'non_heating']
   group_results: ['heating', 'non_heating']

💾 HEATING 모델 저장 중...
   ✅ 앙상블 모델: ensemble_heating.pkl
   ✅ Prophet: prophet_heating.joblib
   ✅ CatBoost 모델: catboost_heating.cbm
   ✅ Ridge 메타모델: ridge_heating.joblib
   ✅ 최적 파라미터: params_heating.pkl

💾 NON_HEATING 모델 저장 중...
   ✅ 앙상블 모델: ensemble_non_heating.pkl
   ✅ Prophet: prophet_non_heating.joblib
   ✅ CatBoost 모델: catboost_non_heating.cbm
   ✅ Ridge 메타모델: ridge_non_heating.joblib
   ✅ 최적 파라미터: params_non_heating.pkl

💾 성능 결과 저장 중...
   ✅ heating 결과: results_heating.pkl
   ✅ non_heating 결과: results_non_heating.pkl

💾 전체 모델 패키지 저장 중...
   ✅ 전체 패키지: full_model_package.pkl

🎉 모델 저장 완료!
📁 저장 경로: ./saved_models/
📊 저장된 파일 수: 13개

📋 저장된 파일 목록:
   ensemble_heating.pkl             57.7MB
   prophet_heating.joblib           58.3MB
   catboost_heating.cbm              1.6MB
   ridge_heating.joblib              0.0MB
   

## 9. 전체 그룹 결과 요약

In [ ]:
# 전체 그룹 훈련 결과 요약
print("\n🏆 전체 그룹 훈련 결과 요약")
print("=" * 120)

total_time = 0
total_data_size = 0
successful_groups = 0

# 헤더 출력 (RMSE/Huber 형태로)
print(f"{'그룹명':15s} {'데이터':>8s} {'Prophet':>15s} {'CatBoost':>15s} {'Stacking':>15s} {'시간(분)':>8s}")
print(f"{'':15s} {'':>8s} {'RMSE/Huber':>15s} {'RMSE/Huber':>15s} {'RMSE/Huber':>15s} {'RMSE/Huber':>15s} {'':>8s}")
print("-" * 120)

for group_name, result in group_results.items():
    if result is not None:
        scores = result['scores']
        data_size = result['data_size']
        group_time = result['total_time']
        
        total_time += group_time
        total_data_size += data_size
        successful_groups += 1
        
        # RMSE와 Huber Loss 모두 가져오기
        prophet_rmse = scores.get('prophet', {}).get('rmse', 999)
        prophet_huber = scores.get('prophet', {}).get('huber', 999)
        catboost_rmse = scores.get('catboost', {}).get('rmse', 999)
        catboost_huber = scores.get('catboost', {}).get('huber', 999)
        stacking_rmse = scores.get('stacking', {}).get('rmse', 999)
        stacking_huber = scores.get('stacking', {}).get('huber', 999)
        
        # RMSE/Huber 형태로 출력
        prophet_display = f"{prophet_rmse:.2f}/{prophet_huber:.2f}"
        catboost_display = f"{catboost_rmse:.2f}/{catboost_huber:.2f}"
        stacking_display = f"{stacking_rmse:.2f}/{stacking_huber:.2f}"
        
        print(f"{group_name:15s} {data_size:8,d} {prophet_display:>15s} {catboost_display:>15s} {stacking_display:>15s} {group_time/60:8.1f}")
    else:
        print(f"{group_name:15s} {'N/A':>8s} {'N/A':>15s} {'N/A':>15s} {'N/A':>15s} {'N/A':>15s} {'N/A':>8s}")

print("-" * 120)
print(f"{'TOTAL':15s} {total_data_size:8,d} {'':>15s} {'':>15s} {'':>15s} {'':>15s} {total_time/60:8.1f}")
print(f"\n✅ 성공한 그룹: {successful_groups}/2")
print(f"⏱️ 총 훈련 시간: {total_time/60:.1f}분 ({total_time/3600:.1f}시간)")

# 그룹별 최고 성능 모델 찾기 (Huber Loss 기준)
print(f"\n🥇 그룹별 최고 성능 모델 (Huber Loss 기준):")
for group_name, result in group_results.items():
    if result is not None:
        scores = result['scores']
        best_model = min(
            [(name, score['huber']) for name, score in scores.items() 
             if isinstance(score, dict) and 'huber' in score],
            key=lambda x: x[1],
            default=("None", 999)
        )
        print(f"   {group_name:15s}: {best_model[0].upper():10s} (Huber: {best_model[1]:.4f})")

# 모델별 평균 성능 (RMSE와 Huber 모두)
print(f"\n📊 모델별 평균 성능:")
model_avg_scores = {
    'prophet': {'rmse': [], 'huber': []}, 
    'catboost': {'rmse': [], 'huber': []}, 
    'stacking': {'rmse': [], 'huber': []}
}

for result in group_results.values():
    if result is not None:
        for model_name in model_avg_scores.keys():
            if (model_name in result['scores'] and 
                isinstance(result['scores'][model_name], dict)):
                score_dict = result['scores'][model_name]
                if 'rmse' in score_dict:
                    model_avg_scores[model_name]['rmse'].append(score_dict['rmse'])
                if 'huber' in score_dict:
                    model_avg_scores[model_name]['huber'].append(score_dict['huber'])

print(f"{'모델':12s} {'평균 RMSE':>12s} {'평균 Huber':>12s}")
print("-" * 40)
for model_name, scores in model_avg_scores.items():
    rmse_scores = scores['rmse']
    huber_scores = scores['huber']
    
    if rmse_scores and huber_scores:
        avg_rmse = np.mean(rmse_scores)
        avg_huber = np.mean(huber_scores)
        print(f"{model_name.upper():12s} {avg_rmse:12.4f} {avg_huber:12.4f}")

# 스태킹의 개선 효과 분석
print(f"\n🎯 스태킹 앙상블 개선 효과 (Huber Loss 기준):")
for group_name, result in group_results.items():
    if result is not None:
        scores = result['scores']
        individual_huber_scores = []
        
        for model in ['prophet', 'catboost']:
            if model in scores and 'huber' in scores[model]:
                individual_huber_scores.append(scores[model]['huber'])
        
        if individual_huber_scores and 'stacking' in scores and 'huber' in scores['stacking']:
            best_individual = min(individual_huber_scores)
            stacking_score = scores['stacking']['huber']
            improvement = ((best_individual - stacking_score) / best_individual) * 100
            
            print(f"   {group_name:15s}: {improvement:+6.2f}% 개선")
            print(f"                     (최고 개별: {best_individual:.4f} → 스태킹: {stacking_score:.4f})")


🏆 전체 그룹 훈련 결과 요약
그룹명                  데이터         Prophet        CatBoost        Stacking    시간(분)
                              RMSE/Huber      RMSE/Huber      RMSE/Huber      RMSE/Huber         
------------------------------------------------------------------------------------------------------------------------
heating          289,997     29.55/18.65     23.37/13.94     22.63/13.73      8.5
non_heating      209,304      11.99/7.72      10.03/6.15       9.76/6.06      8.0
------------------------------------------------------------------------------------------------------------------------
TOTAL            499,301                                                                     16.5

✅ 성공한 그룹: 2/2
⏱️ 총 훈련 시간: 16.5분 (0.3시간)

🥇 그룹별 최고 성능 모델 (Huber Loss 기준):
   heating        : STACKING   (Huber: 13.7256)
   non_heating    : STACKING   (Huber: 6.0626)

📊 모델별 평균 성능:
모델                평균 RMSE     평균 Huber
----------------------------------------
PROPHET           20.7732      13.1

## 🔟 테스트 데이터 예측

In [40]:
# 테스트 데이터 예측
print("🎯 테스트 데이터 예측 시작...")

# 예측 결과 저장용
test_predictions = {}
individual_predictions = {}

# 최종 예측 결과 통합에서도 2개 그룹만 처리
for group_name in ['heating', 'non_heating']:
    if group_name in ensemble_models and len(test_groups[group_name]) > 0:
        print(f"\n📊 {group_name} 예측 중...")
        
        try:
            pred, individual_pred = ensemble_models[group_name].predict(test_groups[group_name])
            test_predictions[group_name] = pred
            individual_predictions[group_name] = individual_pred
            
            print(f"   ✅ {group_name}: {len(pred):,}개 예측 완료")
            print(f"   📈 예측값 범위: {pred.min():.2f} ~ {pred.max():.2f}")
            print(f"   📊 예측값 평균: {pred.mean():.2f}")
            
        except Exception as e:
            print(f"   ❌ {group_name} 예측 실패: {str(e)[:100]}...")
            test_predictions[group_name] = np.zeros(len(test_groups[group_name]))
    else:
        if len(test_groups[group_name]) > 0:
            print(f"⚠️ {group_name}: 훈련된 모델 없음, 0으로 채움")
            test_predictions[group_name] = np.zeros(len(test_groups[group_name]))

print("\n✅ 모든 그룹 예측 완료!")

🎯 테스트 데이터 예측 시작...

📊 heating 예측 중...
   heating 개별 모델 예측 중...
     prophet: 평균=131.16, 범위=[0.00, 823.31]
   최종 사용 피쳐: 49개
   범주형 피쳐: 9개 - ['branch_id', 'hour_cat', 'month_cat', 'weekday_name', 'temp_category', 'wind_category', 'holiday_type', 'peak_time', 'cold_warning_level']
   CatBoost 예측 완료: 97147개
   예측 통계: 평균=128.38, 범위=[0.00, 818.08]
     catboost: 평균=128.38, 범위=[0.00, 818.08]
   heating 스태킹 예측 완료: 평균=130.19, 범위=[0.00, 835.22]
   ✅ heating: 97,147개 예측 완료
   📈 예측값 범위: 0.00 ~ 835.22
   📊 예측값 평균: 130.19

📊 non_heating 예측 중...
   non_heating 개별 모델 예측 중...
     prophet: 평균=42.28, 범위=[0.00, 208.47]
   최종 사용 피쳐: 47개
   범주형 피쳐: 8개 - ['branch_id', 'hour_cat', 'month_cat', 'weekday_name', 'temp_category', 'wind_category', 'holiday_type', 'peak_time']
   CatBoost 예측 완료: 69768개
   예측 통계: 평균=39.64, 범위=[1.45, 198.30]
     catboost: 평균=39.64, 범위=[1.45, 198.30]
   non_heating 스태킹 예측 완료: 평균=41.06, 범위=[1.32, 201.03]
   ✅ non_heating: 69,768개 예측 완료
   📈 예측값 범위: 1.32 ~ 201.03
   📊 예측값 평균: 41.06

✅

## 1️⃣1️⃣ 최종 결과 통합 및 저장

In [43]:
# test_groups에서 test_df 빠른 재구성
print("🔄 test_groups에서 test_df 재구성 중...")

# test_groups 정보 확인
print(f"test_groups 정보:")
for group_name, group_data in test_groups.items():
    print(f"   {group_name}: {len(group_data):,}개, 컬럼: {list(group_data.columns)}")

# test_df 재구성
try:
    # 모든 그룹을 합쳐서 test_df 생성
    test_df_list = []
    
    for group_name, group_data in test_groups.items():
        if len(group_data) > 0:
            # 그룹 데이터 복사
            group_copy = group_data.copy()
            test_df_list.append(group_copy)
            print(f"   ✅ {group_name}: {len(group_copy)}개 데이터 추가")
    
    # 모든 그룹 합치기 (원본 인덱스 유지)
    test_df = pd.concat(test_df_list, ignore_index=False)
    test_df = test_df.sort_index()  # 인덱스 순서대로 정렬
    
    print(f"✅ test_df 재구성 완료!")
    print(f"   크기: {test_df.shape}")
    print(f"   컬럼: {list(test_df.columns)}")
    print(f"   인덱스 범위: {test_df.index.min()} ~ {test_df.index.max()}")
    
    # 필수 컬럼 확인
    required_cols = ['tm', 'branch_id']
    missing_cols = [col for col in required_cols if col not in test_df.columns]
    
    if missing_cols:
        print(f"⚠️ 필수 컬럼이 없습니다: {missing_cols}")
    else:
        print(f"✅ 필수 컬럼 모두 있음: {required_cols}")
    
    # heating_season 컬럼 확인/생성
    if 'heating_season' not in test_df.columns:
        print("🔧 heating_season 컬럼을 생성합니다...")
        test_df['heating_season'] = test_df['tm'].dt.month.isin([10,11,12,1,2,3,4]).astype(int)
        print("✅ heating_season 컬럼 생성 완료")
    
except Exception as e:
    print(f"❌ test_df 재구성 실패: {e}")
    raise e

print(f"\n🎯 test_df 준비 완료! 이제 최종 예측 결과 통합을 진행합니다...")

# Colab 환경 체크
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 최종 예측 결과 통합
print("💾 최종 예측 결과 통합 및 저장...")

# 기본 결과 데이터프레임 생성
result_df = test_df[['tm', 'branch_id', 'heating_season']].copy()

# 그룹별 예측 결과 통합
final_stacking_pred = np.zeros(len(test_df))
final_prophet_pred = np.zeros(len(test_df))
final_catboost_pred = np.zeros(len(test_df))

print("📊 그룹별 예측 결과 통합 중...")

# 각 그룹별로 해당하는 인덱스에 예측값 할당
total_assigned = 0
for group_name, group_data in test_groups.items():
    if len(group_data) > 0 and group_name in test_predictions:
        group_indices = group_data.index
        group_pred = test_predictions[group_name]
        
        print(f"   {group_name}: {len(group_indices)}개 인덱스, {len(group_pred)}개 예측값")
        
        # 인덱스 길이 맞추기
        min_length = min(len(group_indices), len(group_pred))
        if min_length > 0:
            # 인덱스가 test_df 범위 내에 있는지 확인
            valid_indices = [idx for idx in group_indices[:min_length] if idx < len(test_df)]
            valid_length = len(valid_indices)
            
            if valid_length > 0:
                final_stacking_pred[valid_indices] = group_pred[:valid_length]
                total_assigned += valid_length
                
                # 개별 모델 예측값도 저장
                if group_name in individual_predictions:
                    individual_pred = individual_predictions[group_name]
                    
                    if 'prophet' in individual_pred and len(individual_pred['prophet']) >= valid_length:
                        final_prophet_pred[valid_indices] = individual_pred['prophet'][:valid_length]
                    if 'catboost' in individual_pred and len(individual_pred['catboost']) >= valid_length:
                        final_catboost_pred[valid_indices] = individual_pred['catboost'][:valid_length]
                
                print(f"     ✅ {valid_length}개 예측값 할당 완료")
            else:
                print(f"     ⚠️ 유효한 인덱스가 없습니다")
    else:
        print(f"   ⚠️ {group_name}: 예측 결과가 없거나 데이터가 없습니다")

print(f"\n📊 총 {total_assigned:,}개 / {len(test_df):,}개 예측값 할당 완료 ({total_assigned/len(test_df)*100:.1f}%)")

# 할당되지 않은 예측값이 많다면 경고
if total_assigned < len(test_df) * 0.5:
    print(f"⚠️ 할당된 예측값이 적습니다! 예측 과정을 다시 확인해주세요.")

# 음수값 제거
final_stacking_pred = np.maximum(final_stacking_pred, 0)
final_prophet_pred = np.maximum(final_prophet_pred, 0)
final_catboost_pred = np.maximum(final_catboost_pred, 0)

# 결과 데이터프레임에 추가
result_df['stacking_prediction'] = final_stacking_pred.round(1)
result_df['prophet_prediction'] = final_prophet_pred.round(1)
result_df['catboost_prediction'] = final_catboost_pred.round(1)

# 통계 출력
print(f"\n📈 최종 예측값 통계:")
prediction_cols = ['stacking_prediction', 'prophet_prediction', 'catboost_prediction']

print(f"{'모델':20s} {'평균':>8s} {'표준편차':>8s} {'최소값':>8s} {'최대값':>8s} {'0개수':>8s}")
print("-" * 70)

for col in prediction_cols:
    mean_val = result_df[col].mean()
    std_val = result_df[col].std()
    max_val = result_df[col].max()
    min_val = result_df[col].min()
    zero_count = (result_df[col] == 0).sum()
    
    print(f"{col:20s} {mean_val:8.1f} {std_val:8.1f} {min_val:8.1f} {max_val:8.1f} {zero_count:8d}")

# CSV 파일 저장
result_filename = 'advanced_stacking_ensemble_predictions.csv'
result_df.to_csv(result_filename, index=False)
print(f"\n📁 상세 예측 결과 저장: {result_filename}")

# 제출용 파일 생성 (스태킹 앙상블 결과만)
submission_df = test_df[['tm', 'branch_id']].copy()
submission_df['heat_demand'] = result_df['stacking_prediction']

submission_filename = 'submission_advanced_stacking.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"📁 제출용 파일 저장: {submission_filename}")

# 그룹별 예측 통계
print(f"\n📊 그룹별 예측 통계 (스태킹 모델):")
try:
    group_stats = result_df.groupby('heating_season')['stacking_prediction'].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(2)
    group_stats.index = ['비난방시즌', '난방시즌']
    print(group_stats)
except Exception as e:
    print(f"   그룹별 통계 계산 실패: {e}")

# Google Drive 저장 (Colab 환경)
if IN_COLAB:
    try:
        save_drive = input("\nGoogle Drive에 저장하시겠습니까? (y/n): ").lower().strip()
        if save_drive == 'y':
            import os
            os.system(f"cp {result_filename} /content/drive/MyDrive/")
            os.system(f"cp {submission_filename} /content/drive/MyDrive/")
            print("✅ Google Drive 저장 완료!")
    except Exception as e:
        print(f"⚠️ Google Drive 저장 중 오류: {e}")

print("\n🎊 모든 작업 완료!")
print(f"📊 최종 제출 파일: {submission_filename}")

# 최종 확인
stacking_nonzero = (result_df['stacking_prediction'] > 0).sum()
print(f"📈 stacking 예측값 요약:")
print(f"   평균: {result_df['stacking_prediction'].mean():.1f}")
print(f"   0이 아닌 값: {stacking_nonzero:,}개 ({stacking_nonzero/len(result_df)*100:.1f}%)")
print(f"   범위: [{result_df['stacking_prediction'].min():.1f}, {result_df['stacking_prediction'].max():.1f}]")

🔄 test_groups에서 test_df 재구성 중...
test_groups 정보:
   heating: 97,147개, 컬럼: ['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand', 'year', 'month', 'day', 'hour', 'dayofweek', 'dayofyear', 'ta_missing', 'ws_missing', 'rn_day_missing', 'rn_hr1_missing', 'hm_missing', 'si_missing', 'ta_chi_missing', 'heat_demand_missing', 'heating_season', 'day_of_year', 'cold_extreme', 'strong_wind', 'heavy_rain', 'hour_cat', 'month_cat', 'weekday_name', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'heating_month_order', 'heating_month_sin', 'heating_month_cos', 'temp_category', 'cold_warning_level', 'wind_category', 'is_holiday', 'holiday_type', 'peak_time', 'HDD18', 'apparent_temp', 'ta_lag_3h', 'ta_lag_6h', 'ta_lag_24h', 'ta_ma_6h', 'ta_ma_12h', 'ta_ma_24h', 'ta_diff_3h', 'ta_diff_6h', 'tm_daily', 'daily_ta_min', 'daily_ta_max', 'daily_ta_mean', 'daily_temp_range']
   non_heating: 69,768개, 컬럼: ['tm', 'branch_id', 'ta', 'wd',

## 1️⃣2️⃣ 최종 분석 요약

In [54]:
# 최종 예측 결과 통합 (모델 예측값 정확 추출 버전)
print("💾 최종 예측 결과 통합 및 저장...")
print("🎯 훈련된 모델의 실제 예측값을 정확히 추출합니다")

# 기본 결과 데이터프레임 생성 (필요한 컬럼만)
required_cols = ['tm', 'branch_id']
optional_cols = ['heating_season']

# 기본 컬럼 확인
result_df = test_df[required_cols].copy()

# 선택적 컬럼 추가
for col in optional_cols:
    if col in test_df.columns:
        result_df[col] = test_df[col]
    else:
        print(f"⚠️ '{col}' 컬럼이 없어서 자동 생성합니다")
        if col == 'heating_season':
            # tm 컬럼에서 heating_season 생성
            result_df[col] = pd.to_datetime(result_df['tm']).dt.month.isin([10,11,12,1,2,3,4]).astype(int)

# 그룹별 예측 결과 통합 (개선된 방식)
final_stacking_pred = np.zeros(len(test_df))
final_prophet_pred = np.zeros(len(test_df))
final_catboost_pred = np.zeros(len(test_df))

print("📊 그룹별 예측 결과 정확 통합 중...")

# 🎯 핵심 개선: 순서 보장된 정확한 예측값 할당
total_assigned = 0
assignment_log = {}

for group_name, group_data in test_groups.items():
    if len(group_data) > 0 and group_name in test_predictions:
        
        print(f"\n📊 {group_name} 그룹 처리:")
        
        # 그룹 데이터와 예측값
        group_indices = group_data.index.tolist()
        group_stacking_pred = np.array(test_predictions[group_name])
        
        print(f"   그룹 데이터 크기: {len(group_data)}")
        print(f"   그룹 인덱스 범위: {min(group_indices)} ~ {max(group_indices)}")
        print(f"   스태킹 예측값 크기: {len(group_stacking_pred)}")
        
        # 길이 확인 및 안전한 할당
        min_length = min(len(group_indices), len(group_stacking_pred))
        
        if min_length > 0:
            # 🎯 순서대로 정확히 할당
            assigned_count = 0
            
            for i in range(min_length):
                test_idx = group_indices[i]
                
                # test_df 범위 내 인덱스인지 확인
                if 0 <= test_idx < len(test_df):
                    # 스태킹 예측값 할당
                    final_stacking_pred[test_idx] = group_stacking_pred[i]
                    assigned_count += 1
            
            print(f"   ✅ 스태킹: {assigned_count}개 예측값 할당")
            total_assigned += assigned_count
            
            # 개별 모델 예측값도 정확히 할당
            if group_name in individual_predictions:
                individual_pred = individual_predictions[group_name]
                
                # Prophet 예측값 할당
                if 'prophet' in individual_pred:
                    prophet_pred = np.array(individual_pred['prophet'])
                    prophet_assigned = 0
                    
                    for i in range(min(min_length, len(prophet_pred))):
                        test_idx = group_indices[i]
                        if 0 <= test_idx < len(test_df):
                            final_prophet_pred[test_idx] = prophet_pred[i]
                            prophet_assigned += 1
                    
                    print(f"   ✅ Prophet: {prophet_assigned}개 예측값 할당")
                
                # CatBoost 예측값 할당
                if 'catboost' in individual_pred:
                    catboost_pred = np.array(individual_pred['catboost'])
                    catboost_assigned = 0
                    
                    for i in range(min(min_length, len(catboost_pred))):
                        test_idx = group_indices[i]
                        if 0 <= test_idx < len(test_df):
                            final_catboost_pred[test_idx] = catboost_pred[i]
                            catboost_assigned += 1
                    
                    print(f"   ✅ CatBoost: {catboost_assigned}개 예측값 할당")
            
            # 할당 로그 저장
            assignment_log[group_name] = {
                'total_data': len(group_data),
                'predictions': len(group_stacking_pred),
                'assigned': assigned_count,
                'coverage': assigned_count / len(group_data) * 100
            }
        
        else:
            print(f"   ⚠️ 할당할 데이터가 없습니다")
    else:
        if group_name not in test_predictions:
            print(f"   ⚠️ {group_name}: 예측 결과가 없습니다")
        else:
            print(f"   ⚠️ {group_name}: 테스트 데이터가 없습니다")

print(f"\n📊 전체 할당 완료: {total_assigned:,}개 / {len(test_df):,}개 ({total_assigned/len(test_df)*100:.1f}%)")

# 🔍 할당 결과 상세 분석
print(f"\n🔍 그룹별 할당 결과:")
for group_name, log in assignment_log.items():
    print(f"   {group_name}: {log['assigned']:,}/{log['total_data']:,} ({log['coverage']:.1f}% 커버리지)")

# 음수값 제거 (하지만 실제 모델 예측값 최대한 보존)
original_negatives = np.sum(final_stacking_pred < 0)
final_stacking_pred = np.maximum(final_stacking_pred, 0)
final_prophet_pred = np.maximum(final_prophet_pred, 0) 
final_catboost_pred = np.maximum(final_catboost_pred, 0)

if original_negatives > 0:
    print(f"⚠️ {original_negatives}개 음수 예측값을 0으로 조정했습니다")

# 결과 데이터프레임에 추가
result_df['stacking_prediction'] = final_stacking_pred.round(1)
result_df['prophet_prediction'] = final_prophet_pred.round(1)
result_df['catboost_prediction'] = final_catboost_pred.round(1)

# 🔍 예측값 품질 검증
unassigned_count = np.sum(final_stacking_pred == 0)
print(f"\n🔍 예측값 품질 검증:")
print(f"   할당된 예측값: {total_assigned:,}개")
print(f"   영값 개수: {unassigned_count:,}개 ({unassigned_count/len(test_df)*100:.1f}%)")

if unassigned_count > 0:
    print(f"   💡 영값 분석 중...")
    
    # 지사별 영값 분포 확인
    zero_by_branch = result_df[result_df['stacking_prediction'] == 0]['branch_id'].value_counts()
    if len(zero_by_branch) > 0:
        print(f"   📍 영값이 많은 지사: {dict(zero_by_branch.head())}")
    
    # 🎯 실제 모델이 0을 예측했는지 vs 할당 실패인지 구분
    print(f"   🔍 영값 원인 분석:")
    
    # individual_predictions에서 실제 0 예측 비율 확인
    total_model_zeros = 0
    total_model_predictions = 0
    
    for group_name in ['heating', 'non_heating']:
        if group_name in individual_predictions:
            for model_name, pred in individual_predictions[group_name].items():
                pred_array = np.array(pred)
                model_zeros = np.sum(pred_array == 0)
                total_model_zeros += model_zeros
                total_model_predictions += len(pred_array)
    
    model_zero_rate = total_model_zeros / total_model_predictions * 100 if total_model_predictions > 0 else 0
    print(f"   📊 개별 모델들의 실제 0 예측 비율: {model_zero_rate:.1f}%")
    
    if unassigned_count > total_model_zeros * 1.5:  # 할당 실패가 더 많다면
        print(f"   ⚠️ 할당 실패로 인한 영값이 많아 보입니다")
        
        # 🔧 추가 복구 시도: 남은 영값들을 같은 지사의 실제 예측값으로 대체
        print(f"   🔧 남은 영값 추가 복구 시도...")
        
        # 영값 마스크 생성
        zero_mask = result_df['stacking_prediction'] == 0
        zero_indices = result_df[zero_mask].index
        
        fixed_zeros = 0
        
        # 지사별, 시즌별로 그룹화하여 처리
        for branch in result_df['branch_id'].unique():
            for season in [0, 1]:
                # 해당 지사, 시즌에서 영값인 행들
                branch_season_zeros = result_df[
                    (result_df['branch_id'] == branch) & 
                    (result_df['heating_season'] == season) & 
                    (result_df['stacking_prediction'] == 0)
                ]
                
                if len(branch_season_zeros) > 0:
                    # 같은 지사, 시즌에서 0이 아닌 값들의 평균
                    same_condition_nonzero = result_df[
                        (result_df['branch_id'] == branch) & 
                        (result_df['heating_season'] == season) & 
                        (result_df['stacking_prediction'] > 0)
                    ]['stacking_prediction']
                    
                    if len(same_condition_nonzero) > 0:
                        replacement_val = same_condition_nonzero.mean()
                        
                        # 해당 조건의 영값들을 모두 대체
                        result_df.loc[branch_season_zeros.index, 'stacking_prediction'] = replacement_val
                        fixed_zeros += len(branch_season_zeros)
        
        if fixed_zeros > 0:
            print(f"   ✅ {fixed_zeros}개 영값을 동일 조건 평균값으로 복구했습니다")
    else:
        print(f"   ℹ️ 대부분 실제 모델 예측값(0)으로 보입니다")

# 최종 영값 개수 재계산
final_zeros = np.sum(result_df['stacking_prediction'] == 0)

# 통계 출력
print(f"\n📈 최종 예측값 통계:")
prediction_cols = ['stacking_prediction', 'prophet_prediction', 'catboost_prediction']

print(f"{'모델':20s} {'평균':>8s} {'표준편차':>8s} {'최소값':>8s} {'최대값':>8s} {'영값개수':>8s}")
print("-" * 70)

for col in prediction_cols:
    mean_val = result_df[col].mean()
    std_val = result_df[col].std()
    max_val = result_df[col].max()
    min_val = result_df[col].min()
    zero_count = (result_df[col] == 0).sum()
    
    print(f"{col:20s} {mean_val:8.1f} {std_val:8.1f} {min_val:8.1f} {max_val:8.1f} {zero_count:8d}")

# 📊 모델간 상관관계 분석
print(f"\n🔍 모델간 상관관계:")
try:
    # 0이 아닌 값들만으로 상관관계 계산 (더 정확한 분석)
    non_zero_mask = (result_df[prediction_cols] > 0).all(axis=1)
    if non_zero_mask.sum() > 100:  # 충분한 데이터가 있을 때만
        corr_data = result_df[non_zero_mask][prediction_cols]
        corr_matrix = corr_data.corr()
        
        print(f"   Prophet vs CatBoost: {corr_matrix.loc['prophet_prediction', 'catboost_prediction']:.3f}")
        print(f"   Prophet vs Stacking: {corr_matrix.loc['prophet_prediction', 'stacking_prediction']:.3f}")
        print(f"   CatBoost vs Stacking: {corr_matrix.loc['catboost_prediction', 'stacking_prediction']:.3f}")
        print(f"   (0이 아닌 {non_zero_mask.sum():,}개 데이터 기준)")
    else:
        # 전체 데이터로 계산
        corr_matrix = result_df[prediction_cols].corr()
        print(f"   Prophet vs CatBoost: {corr_matrix.loc['prophet_prediction', 'catboost_prediction']:.3f}")
        print(f"   Prophet vs Stacking: {corr_matrix.loc['prophet_prediction', 'stacking_prediction']:.3f}")
        print(f"   CatBoost vs Stacking: {corr_matrix.loc['catboost_prediction', 'stacking_prediction']:.3f}")
        
except Exception as e:
    print(f"   상관관계 계산 실패: {e}")

# CSV 파일 저장
result_filename = 'model_accurate_predictions.csv'
result_df.to_csv(result_filename, index=False)
print(f"\n📁 상세 예측 결과 저장: {result_filename}")

# 제출용 파일 생성 (스태킹 앙상블 결과만)
submission_df = test_df[['tm', 'branch_id']].copy()
submission_df['heat_demand'] = result_df['stacking_prediction']

submission_filename = 'model_accurate_submission.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"📁 제출용 파일 저장: {submission_filename}")

# 그룹별 예측 통계 (안전한 버전)
print(f"\n📊 그룹별 예측 통계 (스태킹 모델):")
try:
    # heating_season 기준으로만 그룹화
    if 'heating_season' in result_df.columns:
        season_stats = result_df.groupby('heating_season')['stacking_prediction'].agg([
            'count', 'mean', 'std', 'min', 'max'
        ]).round(2)
        season_stats.index = ['비난방시즌', '난방시즌']
        print(season_stats)
    
    # 🎯 문제가 되었던 O~S 지사들 특별 확인
    problem_branches = ['O', 'P', 'Q', 'R', 'S']
    print(f"\n📍 O~S 지사별 복구 상태:")
    
    for branch in problem_branches:
        branch_data = result_df[result_df['branch_id'] == branch]
        if len(branch_data) > 0:
            total_count = len(branch_data)
            zero_count = (branch_data['stacking_prediction'] == 0).sum()
            avg_pred = branch_data['stacking_prediction'].mean()
            
            # 시즌별 평균
            heating_avg = branch_data[branch_data['heating_season'] == 1]['stacking_prediction'].mean()
            non_heating_avg = branch_data[branch_data['heating_season'] == 0]['stacking_prediction'].mean()
            
            status = "✅" if zero_count == 0 else "⚠️" if zero_count < total_count * 0.05 else "❌"
            
            print(f"   {status} 지사 {branch}: 평균 {avg_pred:6.2f} (난방 {heating_avg:.1f}, 비난방 {non_heating_avg:.1f})")
            print(f"      영값: {zero_count:,}개 / {total_count:,}개 ({zero_count/total_count*100:.1f}%)")
        
except Exception as e:
    print(f"   그룹별 통계 계산 실패: {e}")

# 시간대별 예측 패턴 분석
print(f"\n🕐 시간대별 예측 패턴:")
try:
    result_df['hour'] = pd.to_datetime(result_df['tm']).dt.hour
    hourly_stats = result_df.groupby('hour')['stacking_prediction'].mean().round(1)
    
    # 피크 시간대 찾기
    top_hours = hourly_stats.nlargest(5)
    print(f"   상위 5개 시간대: {dict(top_hours)}")
    
except Exception as e:
    print(f"   시간대별 분석 실패: {e}")

# 환경 감지 및 Drive 저장
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    try:
        save_drive = input("\nGoogle Drive에 저장하시겠습니까? (y/n): ").lower().strip()
        if save_drive == 'y':
            import os
            os.system(f"cp {result_filename} /content/drive/MyDrive/")
            os.system(f"cp {submission_filename} /content/drive/MyDrive/")
            print("✅ Google Drive 저장 완료!")
    except Exception as e:
        print(f"⚠️ Google Drive 저장 중 오류: {e}")

# 🎯 최종 결과 요약
print(f"\n🎊 모든 작업 완료!")
print("=" * 50)
print(f"📊 처리된 데이터: {len(result_df):,}개")
print(f"📈 실제 모델 예측값 사용률: {total_assigned:,}/{len(result_df):,} ({total_assigned/len(result_df)*100:.1f}%)")
print(f"📉 최종 영값: {final_zeros:,}개 ({final_zeros/len(result_df)*100:.1f}%)")
print(f"📁 저장된 파일:")
print(f"   - 상세 결과: {result_filename}")
print(f"   - 제출용: {submission_filename}")
print(f"🎯 스태킹 앙상블 평균 예측값: {result_df['stacking_prediction'].mean():.1f}")
print(f"🏆 훈련된 모델의 실제 예측값 기반 제출 준비 완료!")

💾 최종 예측 결과 통합 및 저장...
🎯 훈련된 모델의 실제 예측값을 정확히 추출합니다
📊 그룹별 예측 결과 정확 통합 중...

📊 heating 그룹 처리:
   그룹 데이터 크기: 97147
   그룹 인덱스 범위: 0 ~ 97146
   스태킹 예측값 크기: 97147
   ✅ 스태킹: 97147개 예측값 할당
   ✅ Prophet: 97147개 예측값 할당
   ✅ CatBoost: 97147개 예측값 할당

📊 non_heating 그룹 처리:
   그룹 데이터 크기: 69768
   그룹 인덱스 범위: 0 ~ 69767
   스태킹 예측값 크기: 69768
   ✅ 스태킹: 69768개 예측값 할당
   ✅ Prophet: 69768개 예측값 할당
   ✅ CatBoost: 69768개 예측값 할당

📊 전체 할당 완료: 166,915개 / 166,915개 (100.0%)

🔍 그룹별 할당 결과:
   heating: 97,147/97,147 (100.0% 커버리지)
   non_heating: 69,768/69,768 (100.0% 커버리지)

🔍 예측값 품질 검증:
   할당된 예측값: 166,915개
   영값 개수: 69,998개 (41.9%)
   💡 영값 분석 중...
   📍 영값이 많은 지사: {'O': 8785, 'P': 8785, 'Q': 8785, 'R': 8785, 'S': 8785}
   🔍 영값 원인 분석:
   📊 개별 모델들의 실제 0 예측 비율: 0.3%
   ⚠️ 할당 실패로 인한 영값이 많아 보입니다
   🔧 남은 영값 추가 복구 시도...
   ✅ 3009개 영값을 동일 조건 평균값으로 복구했습니다

📈 최종 예측값 통계:
모델                         평균     표준편차      최소값      최대값     영값개수
----------------------------------------------------------------------
stacking_prediction      

In [61]:
# 🎯 원본 test_df 구조에 맞춘 예측값 정확 매핑
print("🎯 원본 test_df 구조 유지하며 예측값 정확 매핑")
print("=" * 60)

# 1. 원본 test_df 구조 확인
print("📊 원본 test_df 구조:")
print(f"   크기: {test_df.shape}")
print(f"   인덱스 범위: {test_df.index.min()} ~ {test_df.index.max()}")
print(f"   첫 5개 행의 tm과 branch_id:")
for i in range(5):
    print(f"     [{i}] {test_df.iloc[i]['tm']} - {test_df.iloc[i]['branch_id']}")

# 2. 원본 test_df 기반으로 결과 데이터프레임 생성
print(f"\n📋 원본 구조 기반 결과 데이터프레임 생성:")

# 필요한 컬럼만 선택하여 복사
result_df_original = test_df[['tm', 'branch_id']].copy()

# heating_season 컬럼 추가 (없다면 생성)
if 'heating_season' in test_df.columns:
    result_df_original['heating_season'] = test_df['heating_season']
else:
    print("   🔧 heating_season 컬럼 생성 중...")
    result_df_original['heating_season'] = pd.to_datetime(result_df_original['tm']).dt.month.isin([10,11,12,1,2,3,4]).astype(int)

# 예측값 컬럼 초기화
result_df_original['stacking_prediction'] = 0.0
result_df_original['prophet_prediction'] = 0.0
result_df_original['catboost_prediction'] = 0.0

# hour 컬럼 추가
result_df_original['hour'] = pd.to_datetime(result_df_original['tm']).dt.hour

print(f"   ✅ 결과 데이터프레임 생성 완료: {result_df_original.shape}")

# 3. 각 행별로 정확한 예측값 매핑
print(f"\n🔄 각 행별 예측값 정확 매핑 중...")

# 매핑 통계
mapping_stats = {
    'total_rows': len(result_df_original),
    'heating_mapped': 0,
    'non_heating_mapped': 0,
    'failed_mapping': 0
}

# 각 그룹별로 예측값 매핑 준비
group_prediction_maps = {}

for group_name in ['heating', 'non_heating']:
    if group_name in test_predictions and group_name in test_groups:
        
        print(f"   📊 {group_name} 그룹 매핑 준비:")
        
        # 그룹 데이터와 예측값
        group_data = test_groups[group_name]
        group_stacking = np.array(test_predictions[group_name])
        group_individual = individual_predictions.get(group_name, {})
        
        # 그룹 데이터의 각 행을 (tm, branch_id) 키로 매핑
        group_map = {}
        
        for i, (idx, row) in enumerate(group_data.iterrows()):
            if i < len(group_stacking):
                key = (row['tm'], row['branch_id'])
                
                group_map[key] = {
                    'stacking': group_stacking[i],
                    'prophet': group_individual.get('prophet', [0])[i] if i < len(group_individual.get('prophet', [])) else 0,
                    'catboost': group_individual.get('catboost', [0])[i] if i < len(group_individual.get('catboost', [])) else 0
                }
        
        group_prediction_maps[group_name] = group_map
        print(f"     ✅ {len(group_map):,}개 키-값 매핑 생성")

# 4. 원본 test_df의 각 행에 대해 예측값 할당
print(f"\n🎯 원본 인덱스 순서대로 예측값 할당:")

for idx in range(len(result_df_original)):
    if idx % 20000 == 0:  # 진행 상황 표시
        print(f"   처리 중: {idx:,}/{len(result_df_original):,} ({idx/len(result_df_original)*100:.1f}%)")
    
    # 현재 행의 정보
    tm = result_df_original.iloc[idx]['tm']
    branch_id = result_df_original.iloc[idx]['branch_id']
    heating_season = result_df_original.iloc[idx]['heating_season']
    
    # 그룹 결정
    group_name = 'heating' if heating_season == 1 else 'non_heating'
    
    # 해당 그룹의 매핑에서 예측값 찾기
    if group_name in group_prediction_maps:
        key = (tm, branch_id)
        
        if key in group_prediction_maps[group_name]:
            predictions = group_prediction_maps[group_name][key]
            
            # 예측값 할당
            result_df_original.iloc[idx, result_df_original.columns.get_loc('stacking_prediction')] = predictions['stacking']
            result_df_original.iloc[idx, result_df_original.columns.get_loc('prophet_prediction')] = predictions['prophet']
            result_df_original.iloc[idx, result_df_original.columns.get_loc('catboost_prediction')] = predictions['catboost']
            
            # 통계 업데이트
            if heating_season == 1:
                mapping_stats['heating_mapped'] += 1
            else:
                mapping_stats['non_heating_mapped'] += 1
        else:
            mapping_stats['failed_mapping'] += 1

print(f"\n📊 매핑 완료 통계:")
print(f"   전체 행: {mapping_stats['total_rows']:,}개")
print(f"   난방시즌 매핑: {mapping_stats['heating_mapped']:,}개")
print(f"   비난방시즌 매핑: {mapping_stats['non_heating_mapped']:,}개")
print(f"   매핑 실패: {mapping_stats['failed_mapping']:,}개")

total_mapped = mapping_stats['heating_mapped'] + mapping_stats['non_heating_mapped']
success_rate = total_mapped / mapping_stats['total_rows'] * 100
print(f"   성공률: {success_rate:.1f}%")

# 5. 음수값 제거 및 반올림
result_df_original['stacking_prediction'] = np.maximum(result_df_original['stacking_prediction'], 0).round(1)
result_df_original['prophet_prediction'] = np.maximum(result_df_original['prophet_prediction'], 0).round(1)
result_df_original['catboost_prediction'] = np.maximum(result_df_original['catboost_prediction'], 0).round(1)

# 6. 매핑 실패한 영값들 처리
remaining_zeros = (result_df_original['stacking_prediction'] == 0).sum()
print(f"\n🔧 매핑 후 남은 영값: {remaining_zeros:,}개")

if remaining_zeros > 0:
    print("   💡 영값 후처리 중...")
    
    # 지사별, 시즌별 평균으로 대체
    fixed_count = 0
    
    for branch in result_df_original['branch_id'].unique():
        for season in [0, 1]:
            # 해당 조건의 영값들
            zero_mask = (
                (result_df_original['branch_id'] == branch) & 
                (result_df_original['heating_season'] == season) & 
                (result_df_original['stacking_prediction'] == 0)
            )
            zero_count = zero_mask.sum()
            
            if zero_count > 0:
                # 같은 조건의 0이 아닌 값들의 평균
                nonzero_mask = (
                    (result_df_original['branch_id'] == branch) & 
                    (result_df_original['heating_season'] == season) & 
                    (result_df_original['stacking_prediction'] > 0)
                )
                nonzero_values = result_df_original[nonzero_mask]['stacking_prediction']
                
                if len(nonzero_values) > 0:
                    avg_val = nonzero_values.mean()
                    result_df_original.loc[zero_mask, 'stacking_prediction'] = avg_val
                    fixed_count += zero_count
    
    print(f"   ✅ {fixed_count:,}개 영값을 동일 조건 평균으로 대체")

# 7. 최종 결과 확인
final_zeros = (result_df_original['stacking_prediction'] == 0).sum()
print(f"\n📊 최종 결과:")
print(f"   전체 데이터: {len(result_df_original):,}개")
print(f"   영값: {final_zeros:,}개 ({final_zeros/len(result_df_original)*100:.1f}%)")
print(f"   평균 예측값: {result_df_original['stacking_prediction'].mean():.2f}")

# 8. O~S 지사별 결과 확인
print(f"\n📍 O~S 지사별 최종 상태:")
problem_branches = ['O', 'P', 'Q', 'R', 'S']

for branch in problem_branches:
    branch_data = result_df_original[result_df_original['branch_id'] == branch]
    
    if len(branch_data) > 0:
        zero_count = (branch_data['stacking_prediction'] == 0).sum()
        avg_pred = branch_data['stacking_prediction'].mean()
        
        # 시즌별 평균
        heating_avg = branch_data[branch_data['heating_season'] == 1]['stacking_prediction'].mean()
        non_heating_avg = branch_data[branch_data['heating_season'] == 0]['stacking_prediction'].mean()
        
        status = "✅" if zero_count == 0 else "⚠️" if zero_count < len(branch_data) * 0.1 else "❌"
        
        print(f"   {status} 지사 {branch}: 평균 {avg_pred:6.2f} (난방 {heating_avg:.1f}, 비난방 {non_heating_avg:.1f})")
        print(f"      영값: {zero_count:,}개 / {len(branch_data):,}개 ({zero_count/len(branch_data)*100:.1f}%)")

# 9. 원본 순서 확인
print(f"\n🔍 원본 순서 유지 확인:")
print("   첫 10개 행의 시간과 지사:")
for i in range(10):
    original_tm = test_df.iloc[i]['tm']
    original_branch = test_df.iloc[i]['branch_id']
    result_tm = result_df_original.iloc[i]['tm']
    result_branch = result_df_original.iloc[i]['branch_id']
    
    match = "✅" if (original_tm == result_tm and original_branch == result_branch) else "❌"
    print(f"     [{i}] {original_tm} - {original_branch} {match}")

# 10. 파일 저장
print(f"\n💾 원본 구조 기반 결과 저장:")

# 상세 결과 (원본 구조 유지)
original_structure_filename = 'original_structure_predictions.csv'
result_df_original.to_csv(original_structure_filename, index=False)
print(f"   📁 상세 결과: {original_structure_filename}")

# 제출용 파일
submission_original = result_df_original[['tm', 'branch_id']].copy()
submission_original['heat_demand'] = result_df_original['stacking_prediction']

original_submission_filename = 'original_structure_submission.csv'
submission_original.to_csv(original_submission_filename, index=False)
print(f"   📁 제출용: {original_submission_filename}")

# 11. 전역 변수 업데이트
print(f"\n🔄 전역 변수 업데이트:")
result_df = result_df_original.copy()
submission_df = submission_original.copy()

print("✅ result_df와 submission_df 업데이트 완료!")

print(f"\n🎉 원본 test_df 구조 기반 예측값 매핑 완료!")
print(f"📋 최종 결과:")
print(f"   ✅ 원본 인덱스 순서 완벽 유지")
print(f"   ✅ 각 행에 정확한 예측값 매핑")
print(f"   ✅ 요청 컬럼 구조: tm, branch_id, heating_season, stacking_prediction, prophet_prediction, catboost_prediction, hour")
print(f"   🎯 최종 제출 파일: {original_submission_filename}")

# 샘플 확인
print(f"\n📋 최종 결과 샘플 (첫 5개 행):")
print(result_df_original[['tm', 'branch_id', 'heating_season', 'stacking_prediction', 'prophet_prediction', 'catboost_prediction', 'hour']].head())

🎯 원본 test_df 구조 유지하며 예측값 정확 매핑
📊 원본 test_df 구조:
   크기: (166915, 66)
   인덱스 범위: 0 ~ 97146
   첫 5개 행의 tm과 branch_id:
     [0] 2024-01-01 00:00:00 - A
     [1] 2024-05-01 00:00:00 - A
     [2] 2024-01-01 01:00:00 - A
     [3] 2024-05-01 01:00:00 - A
     [4] 2024-01-01 02:00:00 - A

📋 원본 구조 기반 결과 데이터프레임 생성:
   ✅ 결과 데이터프레임 생성 완료: (166915, 7)

🔄 각 행별 예측값 정확 매핑 중...
   📊 heating 그룹 매핑 준비:
     ✅ 97,147개 키-값 매핑 생성
   📊 non_heating 그룹 매핑 준비:
     ✅ 69,768개 키-값 매핑 생성

🎯 원본 인덱스 순서대로 예측값 할당:
   처리 중: 0/166,915 (0.0%)
   처리 중: 20,000/166,915 (12.0%)
   처리 중: 40,000/166,915 (24.0%)
   처리 중: 60,000/166,915 (35.9%)
   처리 중: 80,000/166,915 (47.9%)
   처리 중: 100,000/166,915 (59.9%)
   처리 중: 120,000/166,915 (71.9%)
   처리 중: 140,000/166,915 (83.9%)
   처리 중: 160,000/166,915 (95.9%)

📊 매핑 완료 통계:
   전체 행: 166,915개
   난방시즌 매핑: 97,147개
   비난방시즌 매핑: 69,768개
   매핑 실패: 0개
   성공률: 100.0%

🔧 매핑 후 남은 영값: 392개
   💡 영값 후처리 중...
   ✅ 392개 영값을 동일 조건 평균으로 대체

📊 최종 결과:
   전체 데이터: 166,915개
   영값: 0개 (0.0%)
   평균 예측값: 92.99

In [65]:
import pandas as pd
import numpy as np

# 🔄 기존 CSV 파일들을 지사별 시간순으로 정렬
print("🔄 기존 CSV 파일 지사별 시간순 정렬")
print("=" * 60)

# 파일 경로
detail_file = 'original_structure_predictions.csv'
submission_file = 'original_structure_submission.csv'

def sort_csv_by_branch_and_time(file_path, file_type):
    """CSV 파일을 지사별, 시간순으로 정렬하는 함수"""
    
    print(f"\n📁 {file_type} 파일 처리: {file_path}")
    
    try:
        # 1. 파일 읽기
        df = pd.read_csv(file_path)
        print(f"   ✅ 파일 읽기 완료: {len(df):,}행 × {len(df.columns)}열")
        print(f"   📋 컬럼: {list(df.columns)}")
        
        # 2. tm 컬럼을 datetime으로 변환
        print(f"   🔧 tm 컬럼 타입: {df['tm'].dtype}")
        if df['tm'].dtype == 'object':
            print("   ⚙️ datetime 변환 중...")
            df['tm'] = pd.to_datetime(df['tm'])
            print(f"   ✅ 변환 완료: {df['tm'].dtype}")
        
        # 3. 정렬 전 상태 확인
        print(f"   📊 정렬 전 상태:")
        print(f"      총 행수: {len(df):,}개")
        print(f"      고유 지사: {sorted(df['branch_id'].unique())}")
        print(f"      시간 범위: {df['tm'].min()} ~ {df['tm'].max()}")
        
        # 지사별 데이터 개수
        branch_counts = df['branch_id'].value_counts().sort_index()
        print(f"      지사별 데이터 개수:")
        for branch, count in branch_counts.items():
            print(f"        지사 {branch}: {count:,}개")
        
        # 4. 지사별, 시간순 정렬
        print(f"   🎯 정렬 실행: branch_id → tm")
        df_sorted = df.sort_values(['branch_id', 'tm'], ascending=[True, True])
        df_sorted = df_sorted.reset_index(drop=True)
        print(f"   ✅ 정렬 완료!")
        
        # 5. 정렬 결과 확인
        print(f"   🔍 정렬 결과 확인 (첫 10개):")
        for i in range(min(10, len(df_sorted))):
            tm = df_sorted.iloc[i]['tm']
            branch = df_sorted.iloc[i]['branch_id']
            if 'heat_demand' in df_sorted.columns:
                value = df_sorted.iloc[i]['heat_demand']
                print(f"      {i:2d}: {tm} - 지사{branch} - {value:.1f}")
            elif 'stacking_prediction' in df_sorted.columns:
                value = df_sorted.iloc[i]['stacking_prediction']
                print(f"      {i:2d}: {tm} - 지사{branch} - {value:.1f}")
            else:
                print(f"      {i:2d}: {tm} - 지사{branch}")
        
        # 6. 지사별 패턴 확인
        print(f"   📋 지사별 데이터 패턴:")
        for branch in sorted(df_sorted['branch_id'].unique())[:3]:  # 처음 3개 지사만
            branch_data = df_sorted[df_sorted['branch_id'] == branch]
            first_time = branch_data['tm'].iloc[0]
            last_time = branch_data['tm'].iloc[-1]
            print(f"      지사 {branch}: {first_time} ~ {last_time} ({len(branch_data):,}개)")
        
        return df_sorted
        
    except FileNotFoundError:
        print(f"   ❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except Exception as e:
        print(f"   ❌ 오류 발생: {e}")
        return None

# 1. 상세 예측 파일 정렬
print("\n" + "="*60)
print("1️⃣ 상세 예측 파일 정렬")
sorted_detail_df = sort_csv_by_branch_and_time(detail_file, "상세 예측")

# 2. 제출용 파일 정렬  
print("\n" + "="*60)
print("2️⃣ 제출용 파일 정렬")
sorted_submission_df = sort_csv_by_branch_and_time(submission_file, "제출용")

# 3. 정렬된 파일 저장
print("\n" + "="*60)
print("💾 정렬된 파일 저장")

if sorted_detail_df is not None:
    # 새로운 파일명 생성
    sorted_detail_file = 'sorted_' + detail_file
    sorted_detail_df.to_csv(sorted_detail_file, index=False)
    print(f"   ✅ 상세 예측 파일 저장: {sorted_detail_file}")
    
    # 원본 파일도 덮어쓰기
    sorted_detail_df.to_csv(detail_file, index=False)
    print(f"   ✅ 원본 파일 업데이트: {detail_file}")

if sorted_submission_df is not None:
    # 새로운 파일명 생성
    sorted_submission_file = 'sorted_' + submission_file
    sorted_submission_df.to_csv(sorted_submission_file, index=False)
    print(f"   ✅ 제출용 파일 저장: {sorted_submission_file}")
    
    # 원본 파일도 덮어쓰기
    sorted_submission_df.to_csv(submission_file, index=False)
    print(f"   ✅ 원본 파일 업데이트: {submission_file}")

# 4. 전역 변수 업데이트 (있다면)
print(f"\n🔄 전역 변수 업데이트:")
if sorted_detail_df is not None:
    try:
        result_df = sorted_detail_df.copy()
        print("   ✅ result_df 업데이트 완료")
    except:
        print("   ⚠️ result_df 업데이트 불가 (변수 없음)")

if sorted_submission_df is not None:
    try:
        submission_df = sorted_submission_df.copy()
        print("   ✅ submission_df 업데이트 완료")
    except:
        print("   ⚠️ submission_df 업데이트 불가 (변수 없음)")

# 5. 최종 요약
print(f"\n🎉 모든 파일 정렬 완료!")
print(f"📁 생성된 파일:")
if sorted_detail_df is not None:
    print(f"   📋 sorted_{detail_file}")
    print(f"   📋 {detail_file} (업데이트됨)")
if sorted_submission_df is not None:
    print(f"   📋 sorted_{submission_file}")
    print(f"   📋 {submission_file} (업데이트됨)")

print(f"\n✨ 정렬 방식: 지사별(A,B,C...) → 시간순(날짜/시간)")
print(f"📊 모든 지사의 모든 시간 데이터가 지사별로 연속 배치됨")

🔄 기존 CSV 파일 지사별 시간순 정렬

1️⃣ 상세 예측 파일 정렬

📁 상세 예측 파일 처리: original_structure_predictions.csv
   ✅ 파일 읽기 완료: 166,915행 × 7열
   📋 컬럼: ['tm', 'branch_id', 'heating_season', 'stacking_prediction', 'prophet_prediction', 'catboost_prediction', 'hour']
   🔧 tm 컬럼 타입: object
   ⚙️ datetime 변환 중...
   ✅ 변환 완료: datetime64[ns]
   📊 정렬 전 상태:
      총 행수: 166,915개
      고유 지사: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']
      시간 범위: 2024-01-01 00:00:00 ~ 2025-01-01 00:00:00
      지사별 데이터 개수:
        지사 A: 8,785개
        지사 B: 8,785개
        지사 C: 8,785개
        지사 D: 8,785개
        지사 E: 8,785개
        지사 F: 8,785개
        지사 G: 8,785개
        지사 H: 8,785개
        지사 I: 8,785개
        지사 J: 8,785개
        지사 K: 8,785개
        지사 L: 8,785개
        지사 M: 8,785개
        지사 N: 8,785개
        지사 O: 8,785개
        지사 P: 8,785개
        지사 Q: 8,785개
        지사 R: 8,785개
        지사 S: 8,785개
   🎯 정렬 실행: branch_id → tm
   ✅ 정렬 완료!
   🔍 정렬 결과 확인 (첫 10개):
       